# Social Pulse — Análise de Engajamento e Performance de Conteúdo

🇧🇷 Este notebook contém o pipeline completo — carregamento, limpeza, engenharia de features, visualizações, modelagem e export — organizado célula por célula com explicações.

🇬🇧 This notebook contains the complete pipeline — loading, cleaning, feature engineering, visualizations, modeling and export — organized cell by cell with explanations.


<span style="font-size:10px">
<b>## Sumário / Table of Contents</b><br>
🇧🇷 1. Importação de bibliotecas / Loading libraries <br>
🇬🇧 1. Library Import: Essential packages for data science and analysis.<br>
🇧🇷 2. Carregamento do dataset / Leitura dos dados<br>
🇬🇧 2. Dataset Loading: Import and read the dataset correctly, ensuring to skip metadata and set headers.<br>
🇧🇷 3. Inspeção inicial de dados (shape, head, info)<br>
🇬🇧 3. Initial Data Inspection: Explore data shape, basic info, and sample values.<br>
🇧🇷 4. Limpeza dos dados: ajuste de header, remoção de colunas vazias<br>
🇬🇧 4. Data Cleaning: Promote header row, remove empty columns for analysis readiness.<br>
🇧🇷 5. Padronização de marcadores de presença<br>
🇬🇧 5. Attendance Standardization: Harmonize presence markers for consistency.<br>
🇧🇷 6. Engenharia de features (ex: attendance_rate, avg_score, engagement_score)<br>
🇬🇧 6. Feature Engineering: Create relevant metrics such as attendance_rate, avg_score, engagement_score.<br>
🇧🇷 7. Visualização de dados (histogramas, heatmap, séries temporais)<br>
🇬🇧 7. Data Visualization: Generate histograms, heatmaps, and time series to explore trends.<br>
🇧🇷 8. Preparação para modelagem e treinamento (ex: RandomForest com cross-validation)<br>
🇬🇧 8. Modeling Preparation & Training: Prepare data and train models with validation.<br>
🇧🇷 9. Interpretação dos resultados (feature importance)<br>
🇬🇧 9. Interpretation: Analyze feature importance and modeling outcomes.<br>
🇧🇷 10. Exportação dos resultados<br>
🇬🇧 10. Results Export: Save outputs for further use and reporting.<br>
🇧🇷 11. Conclusões e recomendações finais<br>
🇬🇧 11. Conclusions and Recommendations: Summarize insights and actionable steps.<br>
</span>


## 1) Importação de bibliotecas / Import libraries

🇧🇷 Importamos as bibliotecas essenciais para manipulação, análise e visualização de dados.

🇬🇧 We import the essential libraries for data manipulation, analysis and visualization.


In [ ]:
%pip install pandas numpy matplotlib seaborn scikit-learn openpyxl -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn for modeling
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Display settings
pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 200)

print('Libraries installed (if needed) and imported')

Note: you may need to restart the kernel to use updated packages.
Libraries installed (if needed) and imported


## 2) Carregamento do dataset / Load dataset

🇧🇷 Lemos o arquivo Excel fornecido e mostramos as primeiras linhas.

🇬🇧 We read the provided Excel file and show the first rows.


In [ ]:
# Adjust the path if needed
excel_path = r'/mnt/data/Base anonimizada - Eric - PUC-SP (1).xlsx'

try:
    df_raw = pd.read_excel(excel_path, header=None, engine='openpyxl')
    print('Loaded excel with shape:', df_raw.shape)
    display(df_raw.head(8))
except Exception as e:
    print('Error loading excel:', e)
    df_raw = None

Error loading excel: [Errno 2] No such file or directory: '/mnt/data/Base anonimizada - Eric - PUC-SP (1).xlsx'


## 3) Inspeção inicial e promoção da 2ª linha para header / Initial inspection and promote 2nd row to header

🇧🇷 Muitas planilhas possuem linhas de metadados no topo. Aqui promovemos a 2ª linha (index 1) como header.

🇬🇧 Many spreadsheets have metadata rows at the top. Here we promote the 2nd row (index 1) as header.


In [ ]:
# Promote second row to header (header=1) and create working dataframe
if df_raw is not None:
    df = pd.read_excel(excel_path, header=1, engine='openpyxl')
    # remove fully empty columns
    df = df.loc[:, df.notna().any(axis=0)]
    df.columns = df.columns.astype(str).str.strip()
    print('Working df shape:', df.shape)
    display(df.head())
else:
    print('df_raw is None — cannot promote header')

df_raw is None — cannot promote header


## 4) Estrutura do dataset e valores nulos / Dataset structure and null values

🇧🇷 Verificamos info() e contagem de valores nulos para entender limpeza necessária.

🇬🇧 We check info() and count null values to understand necessary cleaning.


In [ ]:
if 'df' in globals():
    display(df.info())
    display(df.isnull().sum().sort_values(ascending=False).head(40))
else:
    print('Working dataframe not available')

Working dataframe not available


# 5) def detectar_colunas_presenca(df):

    🇧🇷 Identifica colunas de presença/aulas usando heurísticas: nomes com 'Aula', datas ou padrões similares.

    🇬🇧 Detects attendance/lesson columns using heuristic rules: names with 'Aula', dates, or similar patterns.

    heuristica = ['/', '-', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']
    colunas_presenca = [c for c in df.columns if ('Aula' in c or any(x in c for x in heuristica))]
    return colunas_presenca


In [ ]:
if 'df' in globals():
    # Heuristic: column names containing 'Aula' or that look like dates (contain '-' or '/'), or numeric-like short names
    attendance_cols = [c for c in df.columns if ('Aula' in c or any(x in c for x in ['/', '-', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']))]
    # Limit to reasonable number if heuristic overselects
    attendance_cols = attendance_cols[:60]
    print('Detected attendance-like columns (sample):', len(attendance_cols))
    display(attendance_cols[:40])
else:
    print('df not found')

df not found


## 6) Padronizar marcadores de presença para numérico (0/1/0.5)

🇧🇷 Mapeamos símbolos comuns (por exemplo '√', 'P', 'N', emoticons) para valores numéricos.

🇬🇧 We map common symbols (e.g. '√', 'P', 'N', emoticons) to numeric values.


In [ ]:
presence_map = {
    '√': 1, 'P': 1, 'Present': 1, 'Yes': 1, 'Sim': 1,
    'N': 0, 'No': 0, 'Absent': 0, '': np.nan, 'nan': np.nan,
    ':-D': 1, ':-)': 1, ':-/': 0, '1/2': 0.5, 'N/A': np.nan
}

# Apply mapping to the detected attendance columns
if 'attendance_cols' in globals() and len(attendance_cols)>0:
    for col in attendance_cols:
        # create a numeric column next to original
        new_col = col + '_att_num'
        df[new_col] = df[col].astype(str).map(presence_map)
    print('Created numeric attendance columns sample:')
    display([c for c in df.columns if c.endswith('_att_num')][:20])
else:
    print('No attendance-like columns detected')

No attendance-like columns detected


## 7) Feature engineering — attendance_rate, avg_score, engagement_score

🇧🇷 Agrupamos indicadores de presença e notas para criar proxies de engajamento e performance.

🇬🇧 We group attendance indicators and grades to create engagement and performance proxies.


In [ ]:
# Identify generated attendance numeric cols
# Ensure 'df' exists (fallback to df_cleaned or reload from excel_path) and then find attendance numeric cols
if 'df' not in globals():
    if 'df_cleaned' in globals():
        df = df_cleaned
        print("Warning: 'df' not found — using 'df_cleaned' as df")
    else:
        try:
            df = pd.read_excel(excel_path, header=1, engine='openpyxl')
            df = df.loc[:, df.notna().any(axis=0)]
            df.columns = df.columns.astype(str).str.strip()
            print("Loaded df from excel_path")
        except Exception as e:
            print("Could not create 'df':", e)
            df = pd.DataFrame()

att_cols = [c for c in df.columns if str(c).endswith('_att_num')]
print('Number of att numeric cols:', len(att_cols))

if len(att_cols)>0:
    df['attendance_count'] = df[att_cols].notna().sum(axis=1)
    df['attendance_sum'] = df[att_cols].sum(axis=1)
    df['attendance_rate'] = df['attendance_sum'] / len(att_cols)
else:
    df['attendance_count'] = 0
    df['attendance_sum'] = 0
    df['attendance_rate'] = np.nan

# Try to detect score-like columns
score_cols = [c for c in df.columns if any(k in c.lower() for k in ['nota','score','grade','prova','avaliacao','av'])]
print('Detected score-like columns (sample):', score_cols[:8])
if len(score_cols)>0:
    df[score_cols] = df[score_cols].apply(pd.to_numeric, errors='coerce')
    df['avg_score'] = df[score_cols].mean(axis=1)
else:
    df['avg_score'] = np.nan

# Engagement composite (simple scaled combo)
# scale avg_score to 0..1 if exists
if 'avg_score' in df.columns and df['avg_score'].notna().any():
    # Min-max scale
    minv = df['avg_score'].min()
    maxv = df['avg_score'].max()
    if pd.notna(minv) and pd.notna(maxv) and maxv>minv:
        df['avg_score_scaled'] = (df['avg_score'] - minv) / (maxv - minv)
    else:
        df['avg_score_scaled'] = 0
else:
    df['avg_score_scaled'] = 0

# engagement_score using attendance_rate (weight 0.6) and avg_score_scaled (weight 0.4)
df['engagement_score'] = df['attendance_rate'].fillna(0)*0.6 + df['avg_score_scaled'].fillna(0)*0.4

display(df[['attendance_rate','avg_score','avg_score_scaled','engagement_score']].head())

Number of att numeric cols: 0
Detected score-like columns (sample): ['avg_score', 'avg_score_scaled', 'engagement_score']


,attendance_rate,avg_score,avg_score_scaled,engagement_score


## 8) Visualizations — histograms, correlation heatmap, time series (aggregate)

🇧🇷 Geramos gráficos-chave para compreender distribuição e relações entre features.

🇬🇧 We generate key graphs to understand distribution and relationships between features.


/Users/fabicampanari/Desktop/Code-academic_performance_pipeline_AI-ML/.venv/lib/python3.12/site-packages/seaborn/matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/Users/fabicampanari/Desktop/Code-academic_performance_pipeline_AI-ML/.venv/lib/python3.12/site-packages/seaborn/matrix.py:207: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: engagement_score, dtype: float64

In [ ]:
# Histogram of attendance_rate
plt.figure(figsize=(10,4))
sns.histplot(df['attendance_rate'].dropna(), kde=True)
plt.title('Distribuição de attendance_rate')
plt.xlabel('attendance_rate')
plt.show()

# Heatmap of correlations among numeric engineered features
num_feats = [f for f in ['attendance_rate','attendance_sum','attendance_count','avg_score','avg_score_scaled','engagement_score'] if f in df.columns]
if len(num_feats)>1:
    plt.figure(figsize=(8,6))
    sns.heatmap(df[num_feats].corr(), annot=True, fmt='.2f')
    plt.title('Heatmap de correlações')
    plt.show()
else:
    print('Not enough numeric features for heatmap')

# Time-series-like aggregation: if there are date-like columns we could aggregate per date (heuristic)
# Here we aggregate engagement_score distribution summary
display(df['engagement_score'].describe())

## 9) Model Preparation and Training (RandomForest with Cross-Validation)

🇧🇷 Detectamos automaticamente colunas candidatas ao alvo (status/resultado) e treinamos um classificador se houver.

🇬🇧 We automatically detect candidate target columns (status/result) and train a classifier if available.


In [ ]:
# Detect potential target columns
candidates = [c for c in df.columns if any(x in c.lower() for x in ['status','resultado','final','aprov','reprov','situacao','situação'])]
print('Target candidates found:', candidates[:8])

target_col = None
for cand in candidates:
    # ensure not to pick engineered columns
    if not cand.endswith('_att_num'):
        target_col = cand
        break

if target_col is None:
    print('No obvious target column found. You may set target_col manually.')
else:
    print('Using target column:', target_col)
    df[target_col] = df[target_col].astype(str)
    le = LabelEncoder()
    df['target_enc'] = le.fit_transform(df[target_col])

    features = [f for f in ['attendance_rate','avg_score','engagement_score'] if f in df.columns]
    X = df[features].fillna(0)
    y = df['target_enc']

    if len(X)>10 and len(set(y))>1:
        model = RandomForestClassifier(n_estimators=200, random_state=42)
        model.fit(X, y)
        print('Train acc:', model.score(X,y))
        scores = cross_val_score(model, X, y, cv=5)
        print('Cross-val scores mean:', scores.mean())
    else:
        print('Not enough data or single-class target for modeling')

Target candidates found: []
No obvious target column found. You may set target_col manually.


## 10) Interpretação / Interpretation — Feature Importance / Importância das Features

🇧🇷 Mostramos a importância das features sempre que o modelo foi treinado.

🇬🇧 We show the importance of features whenever the model was trained.


In [ ]:
if 'model' in globals() and 'X' in globals():
    importances = model.feature_importances_
    feat_imp = pd.Series(importances, index=X.columns).sort_values(ascending=False)
    display(feat_imp)
    plt.figure(figsize=(6,3))
    sns.barplot(x=feat_imp.values, y=feat_imp.index)
    plt.title('Feature importances')
    plt.show()
else:
    print('Model not trained — skip feature importance')

## 11) Exportar resultados

🇧🇷 Exportamos CSV com features geradas e predições (se houver).

🇬🇧 We export CSV with generated features and predictions (if available).


In [ ]:
import os

out_csv = '/mnt/data/social_pulse_engineered_predictions.csv'
if 'df' in globals():
    # include prediction if available
    if 'model' in globals() and 'X' in globals():
        df['predicted'] = model.predict(X.fillna(0))

    out_dir = os.path.dirname(out_csv)
    use_out_csv = out_csv

    if out_dir:
        try:
            # If directory exists, ensure it's writable; otherwise try to create it.
            if os.path.exists(out_dir):
                if not os.access(out_dir, os.W_OK):
                    raise PermissionError(f"Directory not writable: {out_dir}")
            else:
                os.makedirs(out_dir, exist_ok=True)
        except Exception as e:
            # Fallback to current working directory if /mnt is read-only or cannot be created
            fallback = os.path.join(os.getcwd(), os.path.basename(out_csv))
            print(f"Cannot create/write to '{out_dir}': {e}. Falling back to '{fallback}'")
            use_out_csv = fallback

    try:
        df.to_csv(use_out_csv, index=False)
        print('Saved:', use_out_csv)
    except Exception as e:
        print("Error saving CSV:", e)
else:
    print('df not available — nothing exported')

Cannot create/write to '/mnt/data': [Errno 30] Read-only file system: '/mnt'. Falling back to '/Users/fabicampanari/Desktop/Code-academic_performance_pipeline_AI-ML/social_pulse_engineered_predictions.csv'
Saved: /Users/fabicampanari/Desktop/Code-academic_performance_pipeline_AI-ML/social_pulse_engineered_predictions.csv


## 12) Conclusões e próximos passos

🇧🇷 - Criamos proxies de engajamento a partir do dataset acadêmico.
🇬🇧 - Created engagement proxies from the academic dataset.

🇧🇷 - Recomenda-se validar o mapeamento de marcadores manualmente e ajustar pesos do `engagement_score`.
🇬🇧 - It is recommended to manually validate the marker mappings and adjust `engagement_score` weights.

🇧🇷 - Próximos passos: dashboards, explicações SHAP, integração com dados reais de social media.
🇬🇧 - Next steps: dashboards, SHAP explanations, integration with real social media data.


## Carregar e organizar o dataset

🇧🇷 O primeiro passo é carregar e organizar o dataset acadêmico.

🇬🇧 The first step is to load and organize the academic dataset.


In [ ]:
# Robust loading of the Excel file: try known paths and search for the file if needed.
# Do not re-import pandas if already available in the notebook session.
if 'pd' not in globals():
	import pandas as pd

# Import helpers only if not already present
if 'os' not in globals():
	import os
if 'glob' not in globals():
	import glob

# Candidate paths (include the excel_path variable if present)
candidates = []
if 'excel_path' in globals() and excel_path:
	candidates.append(excel_path)
# common locations used in different environments
candidates += [
	'/content/Base anonimizada - Eric - PUC-SP (1).xlsx',
	'/mnt/data/Base anonimizada - Eric - PUC-SP (1).xlsx'
]
# also search the current working directory and /mnt/data for any .xlsx that looks like the dataset
candidates += glob.glob(os.path.join(os.getcwd(), '*.xlsx'))
candidates += glob.glob('/mnt/data/*.xlsx')

# remove duplicates while preserving order
seen = set()
candidates = [p for p in candidates if not (p in seen or seen.add(p))]

found_path = None
for p in candidates:
	if p and os.path.exists(p):
		found_path = p
		break

# try a recursive search by filename pattern as a last resort
if found_path is None:
	matches = glob.glob('**/*Base*anonim*.xlsx', recursive=True)
	if matches:
		found_path = matches[0]

if found_path is None:
	print("Excel file not found. Tried the following candidate paths:")
	for p in candidates:
		print(" -", p)
	print("\nPlease upload the Excel file to the notebook environment (e.g., /mnt/data) or set the variable `excel_path` to its location, then re-run this cell.")
	df = pd.DataFrame()  # keep df defined to avoid NameError downstream
else:
	print("Loading Excel from:", found_path)
	try:
		# read with openpyxl engine for .xlsx
		df = pd.read_excel(found_path, engine='openpyxl')
		print("Loaded shape:", df.shape)

		# Display basic views
		display(df.head())
		display(df.info())
		display(df.isnull().sum())

		# Normalize column names
		df.columns = df.columns.astype(str).str.lower().str.replace(' ', '_')
		display(df.head())
	except Exception as e:
		print("Error loading Excel file:", e)
		df = pd.DataFrame()

Loading Excel from: /Users/fabicampanari/Desktop/Code-academic_performance_pipeline_AI-ML/Base anonimizada - Eric - PUC-SP (1).xlsx
Loaded shape: (1979, 128)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Aula 1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Aula 2,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Aula 3,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Aula 4,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Aula 5,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Aula 6,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Aula 7,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Aula 8,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Aula 9,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Aula 10,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Aula 11,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Aula 12,...,Unnamed: 68,Aula 14,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127
0,NaN,NaN,NaN,NaN,8-fev.-2025,NaN,NaN,NaN,NaN,15-fev.-2025,NaN,NaN,NaN,NaN,22-fev.-2025,NaN,NaN,NaN,NaN,8-mar.-2025,NaN,NaN,NaN,NaN,15-mar.-2025,NaN,NaN,NaN,NaN,29-mar.-2025,NaN,NaN,NaN,NaN,5-abr.-2025,NaN,NaN,NaN,NaN,12-abr.-2025,NaN,NaN,NaN,NaN,26-abr.-2025,NaN,NaN,NaN,NaN,17-mai.-2025,NaN,NaN,NaN,NaN,24-mai.-2025,NaN,NaN,NaN,NaN,31-mai.-2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-Class,NaN,NaN,NaN,NaN,Presence,NaN,NaN,NaN,NaN,Homework,NaN,NaN,NaN,NaN,Class Participation,NaN,NaN,NaN,NaN,Behavior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,...,Bh,Pre-Class,P,Hw,CP,Bh,O,W,Nota,O,W,Nota,O,W,Nota,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P,A,1/2,Total,%,√,+/-,N,Total,%,:-D,:-/,:-&,Total,%,:-(,:-||,NaN,%,Data de nascimento,Telefone Celular,Telefone Fixo,e-mail CPM,e-mail Pessoal,Nome Completo Responsável,Celular Responsável,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,P,√,:-D,NaN,N,P,N,:-D,NaN,N,P,N,NaN,NaN,N,P,√,:-D,NaN,√,P,N,:-D,NaN,N,A,√,NaN,NaN,N,P,N,:-D,NaN,√,P,√,:-D,NaN,√,P,N,:-D,NaN,N,1/2,√,:-/,NaN,√,...,NaN,NaN,NaN,NaN,NaN,NaN,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,NaN,NaN,Reprovado,NaN
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,P,N,:-D,NaN,√,P,√,:-D,NaN,N,P,√,NaN,NaN,√,P,√,NaN,NaN,√,P,√,:-D,NaN,N,A,√,NaN,NaN,√,P,√,:-D,NaN,N,P,√,:-D,NaN,√,P,√,:-D,NaN,√,1/2,√,:-D,NaN,√,...,NaN,NaN,NaN,NaN,NaN,NaN,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,NaN,NaN,NaN,NaN,NaN,NaN,N

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979 entries, 0 to 1978
Columns: 128 entries, Unnamed: 0 to Unnamed: 127
dtypes: float64(1), object(127)
memory usage: 1.9+ MB


None

Unnamed: 0         1
Unnamed: 1         1
Unnamed: 2         1
Unnamed: 3         1
Aula 1          1977
                ... 
Unnamed: 123      20
Unnamed: 124    1578
Unnamed: 125    1673
Unnamed: 126       1
Unnamed: 127    1805
Length: 128, dtype: int64

,unnamed:_0,unnamed:_1,unnamed:_2,unnamed:_3,aula_1,unnamed:_5,unnamed:_6,unnamed:_7,unnamed:_8,aula_2,unnamed:_10,unnamed:_11,unnamed:_12,unnamed:_13,aula_3,unnamed:_15,unnamed:_16,unnamed:_17,unnamed:_18,aula_4,unnamed:_20,unnamed:_21,unnamed:_22,unnamed:_23,aula_5,unnamed:_25,unnamed:_26,unnamed:_27,unnamed:_28,aula_6,unnamed:_30,unnamed:_31,unnamed:_32,unnamed:_33,aula_7,unnamed:_35,unnamed:_36,unnamed:_37,unnamed:_38,aula_8,unnamed:_40,unnamed:_41,unnamed:_42,unnamed:_43,aula_9,unnamed:_45,unnamed:_46,unnamed:_47,unnamed:_48,aula_10,unnamed:_50,unnamed:_51,unnamed:_52,unnamed:_53,aula_11,unnamed:_55,unnamed:_56,unnamed:_57,unnamed:_58,aula_12,...,unnamed:_68,aula_14,unnamed:_70,unnamed:_71,unnamed:_72,unnamed:_73,unnamed:_74,unnamed:_75,unnamed:_76,unnamed:_77,unnamed:_78,unnamed:_79,unnamed:_80,unnamed:_81,unnamed:_82,unnamed:_83,unnamed:_84,unnamed:_85,unnamed:_86,unnamed:_87,unnamed:_88,unnamed:_89,unnamed:_90,unnamed:_91,unnamed:_92,unnamed:_93,unnamed:_94,unnamed:_95,unnamed:_96,unnamed:_97,unnamed:_98,unnamed:_99,unnamed:_100,unnamed:_101,unnamed:_102,unnamed:_103,unnamed:_104,unnamed:_105,unnamed:_106,unnamed:_107,unnamed:_108,unnamed:_109,unnamed:_110,unnamed:_111,unnamed:_112,unnamed:_113,unnamed:_114,unnamed:_115,unnamed:_116,unnamed:_117,unnamed:_118,unnamed:_119,unnamed:_120,unnamed:_121,unnamed:_122,unnamed:_123,unnamed:_124,unnamed:_125,unnamed:_126,unnamed:_127
0,NaN,NaN,NaN,NaN,8-fev.-2025,NaN,NaN,NaN,NaN,15-fev.-2025,NaN,NaN,NaN,NaN,22-fev.-2025,NaN,NaN,NaN,NaN,8-mar.-2025,NaN,NaN,NaN,NaN,15-mar.-2025,NaN,NaN,NaN,NaN,29-mar.-2025,NaN,NaN,NaN,NaN,5-abr.-2025,NaN,NaN,NaN,NaN,12-abr.-2025,NaN,NaN,NaN,NaN,26-abr.-2025,NaN,NaN,NaN,NaN,17-mai.-2025,NaN,NaN,NaN,NaN,24-mai.-2025,NaN,NaN,NaN,NaN,31-mai.-2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-Class,NaN,NaN,NaN,NaN,Presence,NaN,NaN,NaN,NaN,Homework,NaN,NaN,NaN,NaN,Class Participation,NaN,NaN,NaN,NaN,Behavior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,...,Bh,Pre-Class,P,Hw,CP,Bh,O,W,Nota,O,W,Nota,O,W,Nota,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P,A,1/2,Total,%,√,+/-,N,Total,%,:-D,:-/,:-&,Total,%,:-(,:-||,NaN,%,Data de nascimento,Telefone Celular,Telefone Fixo,e-mail CPM,e-mail Pessoal,Nome Completo Responsável,Celular Responsável,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,P,√,:-D,NaN,N,P,N,:-D,NaN,N,P,N,NaN,NaN,N,P,√,:-D,NaN,√,P,N,:-D,NaN,N,A,√,NaN,NaN,N,P,N,:-D,NaN,√,P,√,:-D,NaN,√,P,N,:-D,NaN,N,1/2,√,:-/,NaN,√,...,NaN,NaN,NaN,NaN,NaN,NaN,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,NaN,NaN,Reprovado,NaN
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,P,N,:-D,NaN,√,P,√,:-D,NaN,N,P,√,NaN,NaN,√,P,√,NaN,NaN,√,P,√,:-D,NaN,N,A,√,NaN,NaN,√,P,√,:-D,NaN,N,P,√,:-D,NaN,√,P,√,:-D,NaN,√,1/2,√,:-D,NaN,√,...,NaN,NaN,NaN,NaN,NaN,NaN,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,NaN,NaN,NaN,NaN,NaN,NaN,N

🇧🇷 A inspeção inicial revela que as duas primeiras linhas contêm metadados e cabeçalhos, não dados reais.

🇬🇧 The initial inspection reveals that the first two rows contain metadata and headers, not actual data.

🇧🇷 A linha 1 traz os nomes das colunas, enquanto a linha 0 traz datas para as colunas 'Aula'.

🇬🇧 Row 1 contains meaningful column names, while row 0 contains dates for 'Aula' columns.

🇧🇷 Carregar novamente os dados, pulando a primeira linha e usando a segunda como cabeçalho. Em seguida, tratarei os valores ausentes considerando o contexto: colunas 'Aula' representam presença, e valores ausentes provavelmente indicam falta, sendo inadequado preenchê-los sem mais informações. Para outras colunas, avaliarei preencher ou remover conforme a quantidade de valores ausentes. Também manterei as datas relevantes da primeira linha, unindo-as se necessário.

🇬🇧 Load the data again, skipping the first row and using the second as headers, then handle the missing values considering the context: 'Aula' columns represent attendance and missing values likely represent absences, so filling them may not be appropriate without more context. For other columns, I will evaluate filling or dropping based on the amount of missing values. I will also keep the relevant date columns from the first row and merge them if necessary.


In [ ]:
# Ensure os is available (import only if not already present)
if 'os' not in globals():
    import os

# Build a prioritized list of candidate paths (use existing variables if present)
file_candidates = []

if 'found_path' in globals() and found_path:
    file_candidates.append(found_path)
if 'excel_path' in globals() and excel_path:
    file_candidates.append(excel_path)

# Common runtime locations (keep as fallback)
file_candidates += [
    '/content/Base anonimizada - Eric - PUC-SP (1).xlsx',
    '/mnt/data/Base anonimizada - Eric - PUC-SP (1).xlsx',
    os.path.join(os.getcwd(), 'Base anonimizada - Eric - PUC-SP (1).xlsx')
]

# If a 'candidates' list exists in the notebook, prefer its entries too (dedupe while preserving order)
if 'candidates' in globals():
    combined = file_candidates + candidates
    # remove duplicates preserving order
    seen = set()
    file_candidates = [p for p in combined if p and not (p in seen or seen.add(p))]

# Find the first existing path
file_path = None
for p in file_candidates:
    try:
        if p and os.path.exists(p):
            file_path = p
            break
    except Exception:
        continue

if file_path is None:
    print("Excel file not found. Tried the following candidate paths:")
    for p in file_candidates:
        print(" -", p)
    print("\nPlease upload the Excel file to the notebook environment (e.g., /mnt/data) or set the variable `excel_path` or `found_path` to its location, then re-run this cell.")
    df = pd.DataFrame()  # keep df defined to avoid NameError downstream
else:
    print("Loading Excel from:", file_path)
    try:
        # Load the Excel file skipping the metadata row and using the next row as header
        df = pd.read_excel(file_path, skiprows=1, header=0, engine='openpyxl')

        # Rename columns for consistency after loading with proper headers
        df.columns = df.columns.str.lower().str.replace(' ', '_')

        # Display the first few rows of the DataFrame after reloading
        display(df.head())

        # Check the data types of each column again
        display(df.info())

        # Check for missing values again
        display(df.isnull().sum())

        # For 'aula_' columns, missing values likely represent absence. I will not fill these for now.
        # For other columns, calculate the percentage of missing values for each column
        missing_percentage = df.isnull().sum() / len(df) * 100
        display(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

        # Drop columns with more than 90% missing values
        columns_to_drop = missing_percentage[missing_percentage > 90].index
        df_cleaned = df.drop(columns=columns_to_drop)

        # Identify numerical and categorical columns (excluding 'aula_' columns)
        numerical_cols = df_cleaned.select_dtypes(include=['number']).columns
        categorical_cols = df_cleaned.select_dtypes(include=['object']).columns.difference(
            [col for col in df_cleaned.columns if 'aula_' in col]
        )

        # Fill numerical columns with the median
        for col in numerical_cols:
            if df_cleaned[col].isnull().any():
                df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())

        # Fill categorical columns with the mode (use 'Missing' if mode is not available)
        for col in categorical_cols:
            if df_cleaned[col].isnull().any():
                modes = df_cleaned[col].mode()
                fill_val = modes[0] if not modes.empty else 'Missing'
                df_cleaned[col] = df_cleaned[col].fillna(fill_val)

        # Display the first few rows of the cleaned DataFrame
        display(df_cleaned.head())

        # Check for missing values in the cleaned DataFrame
        display(df_cleaned.isnull().sum().sum())

    except Exception as e:
        print("Error reading or processing the Excel file:", e)
        df = pd.DataFrame()

Loading Excel from: /Users/fabicampanari/Desktop/Code-academic_performance_pipeline_AI-ML/Base anonimizada - Eric - PUC-SP (1).xlsx


,unnamed:_0,unnamed:_1,unnamed:_2,unnamed:_3,8-fev.-2025,unnamed:_5,unnamed:_6,unnamed:_7,unnamed:_8,15-fev.-2025,unnamed:_10,unnamed:_11,unnamed:_12,unnamed:_13,22-fev.-2025,unnamed:_15,unnamed:_16,unnamed:_17,unnamed:_18,8-mar.-2025,unnamed:_20,unnamed:_21,unnamed:_22,unnamed:_23,15-mar.-2025,unnamed:_25,unnamed:_26,unnamed:_27,unnamed:_28,29-mar.-2025,unnamed:_30,unnamed:_31,unnamed:_32,unnamed:_33,5-abr.-2025,unnamed:_35,unnamed:_36,unnamed:_37,unnamed:_38,12-abr.-2025,unnamed:_40,unnamed:_41,unnamed:_42,unnamed:_43,26-abr.-2025,unnamed:_45,unnamed:_46,unnamed:_47,unnamed:_48,17-mai.-2025,unnamed:_50,unnamed:_51,unnamed:_52,unnamed:_53,24-mai.-2025,unnamed:_55,unnamed:_56,unnamed:_57,unnamed:_58,31-mai.-2025,...,unnamed:_68,unnamed:_69,unnamed:_70,unnamed:_71,unnamed:_72,unnamed:_73,unnamed:_74,unnamed:_75,unnamed:_76,unnamed:_77,unnamed:_78,unnamed:_79,unnamed:_80,unnamed:_81,unnamed:_82,unnamed:_83,unnamed:_84,unnamed:_85,unnamed:_86,pre-class,unnamed:_88,unnamed:_89,unnamed:_90,unnamed:_91,presence,unnamed:_93,unnamed:_94,unnamed:_95,unnamed:_96,homework,unnamed:_98,unnamed:_99,unnamed:_100,unnamed:_101,class_participation,unnamed:_103,unnamed:_104,unnamed:_105,unnamed:_106,behavior,unnamed:_108,unnamed:_109,unnamed:_110,unnamed:_111,unnamed:_112,unnamed:_113,unnamed:_114,unnamed:_115,unnamed:_116,unnamed:_117,unnamed:_118,unnamed:_119,unnamed:_120,unnamed:_121,unnamed:_122,unnamed:_123,unnamed:_124,unnamed:_125,unnamed:_126,unnamed:_127
0,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,P,Hw,CP,Bh,Pre-Class,...,Bh,Pre-Class,P,Hw,CP,Bh,O,W,Nota,O,W,Nota,O,W,Nota,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P,A,1/2,Total,%,√,+/-,N,Total,%,:-D,:-/,:-&,Total,%,:-(,:-||,NaN,%,Data de nascimento,Telefone Celular,Telefone Fixo,e-mail CPM,e-mail Pessoal,Nome Completo Responsável,Celular Responsável,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
1,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,P,√,:-D,NaN,N,P,N,:-D,NaN,N,P,N,NaN,NaN,N,P,√,:-D,NaN,√,P,N,:-D,NaN,N,A,√,NaN,NaN,N,P,N,:-D,NaN,√,P,√,:-D,NaN,√,P,N,:-D,NaN,N,1/2,√,:-/,NaN,√,...,NaN,NaN,NaN,NaN,NaN,NaN,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,NaN,NaN,Reprovado,NaN
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,P,N,:-D,NaN,√,P,√,:-D,NaN,N,P,√,NaN,NaN,√,P,√,NaN,NaN,√,P,√,:-D,NaN,N,A,√,NaN,NaN,√,P,√,:-D,NaN,N,P,√,:-D,NaN,√,P,√,:-D,NaN,√,1/2,√,:-D,NaN,√,...,NaN,NaN,NaN,NaN,NaN,NaN,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,N,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0,0,desistiu,desistiu,0,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1978 entries, 0 to 1977
Columns: 128 entries, unnamed:_0 to unnamed:_127
dtypes: float64(1), object(127)
memory usage: 1.9+ MB


None

unnamed:_0         0
unnamed:_1         0
unnamed:_2         0
unnamed:_3         0
8-fev.-2025     1977
                ... 
unnamed:_123      19
unnamed:_124    1577
unnamed:_125    1672
unnamed:_126       0
unnamed:_127    1804
Length: 128, dtype: int64

8-fev.-2025     99.949444
unnamed:_111    99.949444
unnamed:_43     99.949444
unnamed:_28     99.949444
unnamed:_66     99.949444
                  ...    
unnamed:_5       1.061678
unnamed:_121     0.960566
unnamed:_123     0.960566
unnamed:_109     0.050556
unnamed:_76      0.050556
Length: 92, dtype: float64

,unnamed:_0,unnamed:_1,unnamed:_2,unnamed:_3,unnamed:_5,unnamed:_7,15-fev.-2025,unnamed:_10,unnamed:_11,unnamed:_12,22-fev.-2025,unnamed:_15,unnamed:_16,unnamed:_17,8-mar.-2025,unnamed:_20,unnamed:_21,unnamed:_22,15-mar.-2025,unnamed:_25,unnamed:_26,unnamed:_27,29-mar.-2025,unnamed:_30,unnamed:_31,unnamed:_32,5-abr.-2025,unnamed:_35,unnamed:_36,unnamed:_37,12-abr.-2025,unnamed:_40,unnamed:_41,unnamed:_42,26-abr.-2025,unnamed:_45,unnamed:_46,unnamed:_47,17-mai.-2025,unnamed:_50,unnamed:_51,unnamed:_52,24-mai.-2025,unnamed:_55,unnamed:_56,unnamed:_57,31-mai.-2025,unnamed:_60,unnamed:_61,unnamed:_62,7-jun.-2025,unnamed:_65,unnamed:_67,unnamed:_74,unnamed:_75,unnamed:_76,unnamed:_77,unnamed:_78,unnamed:_79,unnamed:_80,unnamed:_81,unnamed:_82,unnamed:_83,unnamed:_84,unnamed:_85,unnamed:_86,pre-class,unnamed:_88,unnamed:_89,unnamed:_90,unnamed:_91,presence,unnamed:_93,unnamed:_94,unnamed:_95,unnamed:_96,homework,unnamed:_98,unnamed:_99,unnamed:_100,unnamed:_101,class_participation,unnamed:_103,unnamed:_104,unnamed:_105,unnamed:_106,behavior,unnamed:_108,unnamed:_109,unnamed:_110,unnamed:_119,unnamed:_120,unnamed:_121,unnamed:_122,unnamed:_123,unnamed:_124,unnamed:_125,unnamed:_126
0,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,P,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,CP,O,W,Nota,O,W,Nota,O,W,Nota,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P,A,1/2,Total,%,√,+/-,N,Total,%,:-D,:-/,:-&,Total,%,:-(,:-||,11.0,%,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final
1,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,A,:-D,√,P,√,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,N,P,N,:-D,√,P,√,:-D,√,P,N,:-D,N,1/2,√,:-/,√,A,N,:-D,N,P,:-D,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Reprovado
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,A,:-D,√,P,N,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,A,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,1/2,√,:-D,√,A,√,:-D,N,P,:-D,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,A,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,:-D,1900-01-08 00:00:00,10,0,9,10,0,9,10,0,0,0,desistiu,desistiu,0,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,N,:-D,N,P,√,:-D,N,P,N,:-D,√,P,+/-,:-D,√,P,N,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,N,P,N,:-D,N,P,N,:-D,√,P,:-D,1900-01-08 12:00:00,9,8.8,8,9.2,8.72,7.5,8.6,8.16,8.56,7.094233,Aprovado,Ok,5,0,7,12,0.416667,13,0,0,13,1,3,1,7,11,0.318182,8,0,0,8,0.888889,0,0,13.0,1,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado


np.int64(0)

🇧🇷 Os dados foram carregados e a limpeza inicial já ocorreu, removendo colunas com muitos valores ausentes e preenchendo faltantes em algumas colunas.

🇬🇧 The data has been loaded and initial cleaning has taken place, with columns containing too many missing values dropped and missing values in some columns filled.

🇧🇷 A primeira linha do novo DataFrame ainda contém os nomes das colunas originais da segunda linha do arquivo. Vou promover a primeira linha para o cabeçalho real e então remover essa primeira linha.

🇬🇧 The first row of the reloaded DataFrame still contains the original column headers from row 2. I will promote the first row to become the actual header and then drop the original first row.

🇧🇷 Depois, verificarei novamente os valores ausentes para garantir que a limpeza foi feita corretamente.

🇬🇧 Then I will check the missing values again to make sure the cleaning is done properly.


In [ ]:
# Promote the first row to become the header
df_cleaned.columns = df_cleaned.iloc[0]

# Drop the first row as it is now the header
df_cleaned = df_cleaned[1:].reset_index(drop=True)

# Display the first few rows of the DataFrame after setting the correct header
display(df_cleaned.head())

# Check for remaining missing values
display(df_cleaned.isnull().sum().sum())

,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,P,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,CP,O,W,Nota,O,W,Nota,O,W,Nota,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P,A,1/2,Total,%,√,+/-,N,Total,%,:-D,:-/,:-&,Total,%,:-(,:-||,11.0,%,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovações = perda de vaga),Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado),Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, escolha o motivo",Situação Final
0,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,A,:-D,√,P,√,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,N,P,N,:-D,√,P,√,:-D,√,P,N,:-D,N,1/2,√,:-/,√,A,N,:-D,N,P,:-D,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Reprovado
1,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,A,:-D,√,P,N,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,A,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,1/2,√,:-D,√,A,√,:-D,N,P,:-D,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,A,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,:-D,1900-01-08 00:00:00,10,0,9,10,0,9,10,0,0,0,desistiu,desistiu,0,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,N,:-D,N,P,√,:-D,N,P,N,:-D,√,P,+/-,:-D,√,P,N,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,N,P,N,:-D,N,P,N,:-D,√,P,:-D,1900-01-08 12:00:00,9,8.8,8,9.2,8.72,7.5,8.6,8.16,8.56,7.094233,Aprovado,Ok,5,0,7,12,0.416667,13,0,0,13,1,3,1,7,11,0.318182,8,0,0,8,0.888889,0,0,13.0,1,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,5,Estudante 5 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,A,√,:-D,√,P,:-D,9,9.4,9.24,9,9.6,9.36,8.5,7,7.6,8.733333,8.726856,Aprovado,Ok,10,0,2,12,0.833333,11,2,0,11,0.85,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,11.0,1,2025-06-06 18:18:41,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado


np.int64(0)

## Explorar e descrever o dataset

🇧🇷 Analisar as colunas, tipos de dados e valores presentes no dataset para entender seu conteúdo e descrevê-lo.

🇬🇧 Analyze the columns, data types and values present in the dataset to understand its content and describe it.


🇧🇷 Exibir as 5 primeiras linhas do DataFrame `df_cleaned` para obter uma visão geral da sua estrutura e conteúdo.

🇬🇧 Display the first 5 rows of the `df_cleaned` DataFrame to get a general overview of its structure and content.

🇧🇷 Em seguida, exibir as informações sobre o DataFrame, incluindo o número de linhas e colunas, tipos de dados e contagem de valores não nulos para entender suas características.

🇬🇧 Then, display information about the DataFrame including the number of rows and columns, data types, and non-null counts to understand its characteristics.


In [ ]:
display(df_cleaned.head())
display(df_cleaned.info())

,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,P,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,Hw,CP,Pre-Class,P,CP,O,W,Nota,O,W,Nota,O,W,Nota,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P,A,1/2,Total,%,√,+/-,N,Total,%,:-D,:-/,:-&,Total,%,:-(,:-||,11.0,%,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovações = perda de vaga),Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado),Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, escolha o motivo",Situação Final
0,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,A,:-D,√,P,√,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,N,P,N,:-D,√,P,√,:-D,√,P,N,:-D,N,1/2,√,:-/,√,A,N,:-D,N,P,:-D,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Reprovado
1,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,A,:-D,√,P,N,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,A,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,1/2,√,:-D,√,A,√,:-D,N,P,:-D,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,A,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,:-D,1900-01-08 00:00:00,10,0,9,10,0,9,10,0,0,0,desistiu,desistiu,0,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,N,:-D,N,P,√,:-D,N,P,N,:-D,√,P,+/-,:-D,√,P,N,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,N,P,N,:-D,N,P,N,:-D,√,P,:-D,1900-01-08 12:00:00,9,8.8,8,9.2,8.72,7.5,8.6,8.16,8.56,7.094233,Aprovado,Ok,5,0,7,12,0.416667,13,0,0,13,1,3,1,7,11,0.318182,8,0,0,8,0.888889,0,0,13.0,1,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,5,Estudante 5 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,A,√,:-D,√,P,:-D,9,9.4,9.24,9,9.6,9.36,8.5,7,7.6,8.733333,8.726856,Aprovado,Ok,10,0,2,12,0.833333,11,2,0,11,0.85,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,11.0,1,2025-06-06 18:18:41,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 98 columns):
 #   Column                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                         --------------  -----  
 0   Nome Planilha Feedback                                                                                         1977 non-null   object 
 1   Sala                                                                                                           1977 non-null   object 
 2   Num                                                                                                            1977 non-null   object 
 3   NOME COMPLETO                                                                                                  1977 non-null   object 
 4   P                                               

None

🇧🇷 Antes de analisar as colunas categóricas e numéricas, é importante lidar com possíveis nomes de colunas duplicados. Primeiro, identifique as colunas duplicadas e, em seguida, renomeie-as para garantir que sejam únicas.

🇬🇧 Before analyzing the categorical and numerical columns, it's important to address any duplicate column names. First, identify any duplicate columns and then rename them to ensure all column names are unique.

Address the duplicate column names before proceeding with the analysis of categorical and numerical columns. First identify the duplicate columns and then rename them to be unique.


In [ ]:
# Identify duplicate column names
duplicate_columns = df_cleaned.columns[df_cleaned.columns.duplicated(keep=False)]
print("Duplicate columns:", duplicate_columns)

# Rename duplicate columns to make them unique
cols = pd.Series(df_cleaned.columns)
for dup in cols[cols.duplicated()].unique():
    cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
df_cleaned.columns = cols

# Now retry the analysis of categorical and numerical columns
# Identify categorical columns
categorical_cols = df_cleaned.select_dtypes(include=['object']).columns

# Get value counts and frequencies for relevant categorical columns
relevant_categorical_cols = ['Sala', 'Situação aluno no semestre', 'Situação Final_0', 'Status aluno semestre anterior', 'Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)', 'Se aluno não está mais frequentando as aulas, escolha o motivo']
for col in relevant_categorical_cols:
    if col in categorical_cols:
        print(f"\nValue counts for '{col}':")
        display(df_cleaned[col].value_counts())
        print(f"\nValue frequencies for '{col}':")
        display(df_cleaned[col].value_counts(normalize=True))

# Identify numerical columns
numerical_cols = df_cleaned.select_dtypes(include=['number']).columns

# Calculate descriptive statistics for relevant numerical columns
relevant_numerical_cols = ['Quantidade de reprovações no curso (4 reprovações = perda de vaga)']
for col in relevant_numerical_cols:
    if col in numerical_cols:
        print(f"\nDescriptive statistics for '{col}':")
        display(df_cleaned[col].describe())

# Identify potentially important columns for analysis of student performance and marketing impact
important_columns = [
    'Sala',
    'Situação aluno no semestre',
    'Situação Final_0',
    'Status aluno semestre anterior',
    'Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)',
    'Se aluno não está mais frequentando as aulas, escolha o motivo',
    'Quantidade de reprovações no curso (4 reprovações = perda de vaga)'
]

print("\nPotentially important columns for analysis:")
for col in important_columns:
    print(f"- {col}")

Duplicate columns: Index(['P', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP',
       'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'Hw', 'CP', 'Pre-Class', 'P', 'CP', 'O', 'W', 'Nota',
       'O', 'W', 'Nota', 'O', 'W', 'Nota', 'Situação Final', '√', '+/-', 'N', 'Total', '%', 'P', 'Total', '%', '√', '+/-', 'N', 'Total', '%', 'Total', '%', '%', 'Situação Final'],
      dtype='object', name=0)

Value counts for 'Sala':


Sala
Four Corners A1    222
Four Corners C1    209
Four Corners B1    195
Four Corners D1    178
Four Corners E1    161
Four Corners F1    141
Four Corners I1    110
Four Corners G1    110
Four Corners H1    110
Four Corners J1    109
Four Corners D2     56
Four Corners G2     41
Four Corners F2     39
Four Corners A2     38
Four Corners H2     31
Four Corners C2     31
Four Corners I2     28
Four Corners J2     28
Four Corners B2     27
Four Corners E2     21
Four Corners A3     18
Four Corners J3     18
Four Corners H3     17
Four Corners I3     17
Four Corners G3     14
Four Corners B3      8
Name: count, dtype: int64


Value frequencies for 'Sala':


Sala
Four Corners A1    0.112291
Four Corners C1    0.105716
Four Corners B1    0.098634
Four Corners D1    0.090035
Four Corners E1    0.081437
Four Corners F1    0.071320
Four Corners I1    0.055640
Four Corners G1    0.055640
Four Corners H1    0.055640
Four Corners J1    0.055134
Four Corners D2    0.028326
Four Corners G2    0.020738
Four Corners F2    0.019727
Four Corners A2    0.019221
Four Corners H2    0.015680
Four Corners C2    0.015680
Four Corners I2    0.014163
Four Corners J2    0.014163
Four Corners B2    0.013657
Four Corners E2    0.010622
Four Corners A3    0.009105
Four Corners J3    0.009105
Four Corners H3    0.008599
Four Corners I3    0.008599
Four Corners G3    0.007081
Four Corners B3    0.004047
Name: proportion, dtype: float64


Value counts for 'Situação aluno no semestre':


Situação aluno no semestre
evadiu               1750
desistiu               97
mudou de turma         68
ok                     36
trancou matrícula      25
suspenso                1
Name: count, dtype: int64


Value frequencies for 'Situação aluno no semestre':


Situação aluno no semestre
evadiu               0.885180
desistiu             0.049064
mudou de turma       0.034396
ok                   0.018209
trancou matrícula    0.012645
suspenso             0.000506
Name: proportion, dtype: float64


Value counts for 'Status aluno semestre anterior':


Status aluno semestre anterior
Aprovado                  1230
Aluno Novo                 395
Aprovado por Conselho      138
Reprovado                  130
Reprovado por Conselho      37
Ex-aluno(a)                 32
trancou matrícula           12
Aluno novo                   1
Ex aluno                     1
aprovado                     1
Name: count, dtype: int64


Value frequencies for 'Status aluno semestre anterior':


Status aluno semestre anterior
Aprovado                  0.622155
Aluno Novo                0.199798
Aprovado por Conselho     0.069803
Reprovado                 0.065756
Reprovado por Conselho    0.018715
Ex-aluno(a)               0.016186
trancou matrícula         0.006070
Aluno novo                0.000506
Ex aluno                  0.000506
aprovado                  0.000506
Name: proportion, dtype: float64


Value counts for 'Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)':


Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)
Sim    1912
NÃO      65
Name: count, dtype: int64


Value frequencies for 'Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)':


Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)
Sim    0.967122
NÃO    0.032878
Name: proportion, dtype: float64


Value counts for 'Se aluno não está mais frequentando as aulas, escolha o motivo':


Se aluno não está mais frequentando as aulas, escolha o motivo
Sem retorno do estudante                                                                                                         1821
Trabalho                                                                                                                           34
Desinteresse                                                                                                                       21
Saúde                                                                                                                              18
Estudos                                                                                                                            17
Problemas pessoais                                                                                                                 14
Dificuldade                                                                                                                        12


Value frequencies for 'Se aluno não está mais frequentando as aulas, escolha o motivo':


Se aluno não está mais frequentando as aulas, escolha o motivo
Sem retorno do estudante                                                                                                         0.921093
Trabalho                                                                                                                         0.017198
Desinteresse                                                                                                                     0.010622
Saúde                                                                                                                            0.009105
Estudos                                                                                                                          0.008599
Problemas pessoais                                                                                                               0.007081
Dificuldade                                                                                                  


Potentially important columns for analysis:
- Sala
- Situação aluno no semestre
- Situação Final_0
- Status aluno semestre anterior
- Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)
- Se aluno não está mais frequentando as aulas, escolha o motivo
- Quantidade de reprovações no curso (4 reprovações = perda de vaga)


🇧🇷 Exemplo Básico de Chatbot

Aqui está um exemplo simples de como um chatbot baseado em regras pode interagir com o dataset `df_cleaned`. Este exemplo mostra como o chatbot pode responder perguntas sobre a situação final de um aluno com base no nome.

🇺🇸 Basic Chatbot Example

Here is a simple example of how a rule-based chatbot could interact with the `df_cleaned` dataset. This example demonstrates how the chatbot can answer questions about a student's final status based on their name.

**Nota / Note:** Este é um exemplo simplificado. Um chatbot real necessitaria de processamento de linguagem natural mais robusto e uma interface de conversação.

This is a simplified example. A real chatbot would require more robust natural language processing and a conversational interface.


In [ ]:
def chatbot_response(student_name, dataframe):
    """
    Função básica de chatbot para obter a situação final de um aluno.
    """
    # Procurar o aluno no DataFrame (ignorando case para simplificar)
    student_data = dataframe[dataframe['NOME COMPLETO'].str.contains(student_name, case=False, na=False)]

    if not student_data.empty:
        # Se o aluno for encontrado, retornar a situação final
        # Lidar com possíveis múltiplas entradas para o mesmo nome (apenas pegar a primeira)
        final_status = student_data['Situação Final_1'].iloc[0]
        return f"A situação final de {student_name} é: {final_status}"
    else:
        return f"Aluno '{student_name}' não encontrado no dataset."

# Exemplo de uso do chatbot
print(chatbot_response("Estudante 1", df_cleaned))
print(chatbot_response("Estudante 100", df_cleaned)) # Exemplo de aluno não encontrado
print(chatbot_response("Estudante 5", df_cleaned)) # Adicionando mais um exemplo
print(chatbot_response("Estudante 15", df_cleaned)) # Adicionando outro exemplo
print(chatbot_response("Estudante 10", df_cleaned)) # Testando com Estudante 10

A situação final de Estudante 1 é: Reprovado
Aluno 'Estudante 100' não encontrado no dataset.
A situação final de Estudante 5 é: Aprovado
A situação final de Estudante 15 é: desistiu
A situação final de Estudante 10 é: Aprovado


In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # 1. Consider different types of chatbots
print("1. Tipos de Chatbots:")
print("- Rule-based chatbots: Follow predefined rules and scripts. Suitable for answering FAQs based on the dataset (e.g., 'What is my current status?', 'Can I lock my matrícula?'). Limited in handling complex or unpredicted queries.")
print("- AI-powered chatbots (e.g., using Natural Language Processing - NLP): Can understand and respond to more complex, natural language queries. Can potentially analyze sentiment and provide more personalized responses based on student data. Requires more data for training and more complex development.")
print("\nPara este projeto, uma abordagem híbrida pode ser interessante, começando com um chatbot baseado em regras para informações diretas do dataset e explorando a integração de NLP para consultas mais complexas e personalizadas.")

# 2. Brainstorm potential use cases
print("\n2. Casos de Uso Potenciais do Chatbot (alavancando o df_cleaned):")
print("- **Identificação de Alunos em Risco:** O chatbot pode proativamente contatar alunos cujo 'Situação aluno no semestre' ou 'Quantidade de reprovações' indique risco de evasão, oferecendo suporte ou direcionando para recursos.")
print("- **Informações sobre Status Acadêmico:** Responder a perguntas dos alunos sobre seu 'Situação Final_0', 'Status aluno semestre anterior', etc.")
print("- **Suporte Baseado em Motivos de Não Frequência:** Se um aluno indicar um motivo como 'Trabalho' ou 'Problemas de Saúde' ('Se aluno não está mais frequentando as aulas, escolha o motivo'), o chatbot pode oferecer informações sobre flexibilização, apoio psicológico, etc.")
print("- **Lembretes e Prazos:** Enviar notificações sobre prazos de matrícula, trancamento ('Aluno pode trancar matrícula ou manter trancamento?'), etc.")
print("- **Coleta de Feedback:** Realizar pesquisas rápidas ou coletar feedback sobre a experiência do aluno para identificar problemas comuns.")

# 3. Discuss technical feasibility and challenges
print("\n3. Viabilidade Técnica e Desafios:")
print("- **Viabilidade:** É tecnicamente viável integrar um chatbot com um dataset como o df_cleaned usando APIs ou acessando diretamente um banco de dados onde os dados estejam armazenados. Plataformas de desenvolvimento de chatbot (ex: Dialogflow, Rasa, Microsoft Bot Framework) oferecem ferramentas para isso.")
print("- **Desafios:**")
print("  - **Privacidade de Dados:** Acesso e manuseio seguro dos dados do aluno são cruciais e exigem conformidade com leis de proteção de dados (LGPD no Brasil).")
print("  - **Acesso a Dados em Tempo Real:** Para informações atualizadas, o chatbot precisa de acesso a dados em tempo real ou quase real.")
print("  - **Processamento de Linguagem Natural (PNL):** Entender a intenção do aluno e extrair informações relevantes de linguagem natural exige treinamento e ajuste do modelo de PNL.")
print("  - **Escalabilidade:** Garantir que o chatbot possa lidar com um grande número de alunos simultaneamente.")
print("  - **Integração com Sistemas Existentes:** Conectar o chatbot aos sistemas acadêmicos da PUC-SP pode ser complexo.")

# 4. Outline a high-level plan
print("\n4. Plano de Alto Nível para Desenvolvimento/Implementação:")
print("- **Fase 1: Design e Definição de Escopo:** Definir os casos de uso prioritários, o tipo de chatbot e a plataforma a ser utilizada.")
print("- **Fase 2: Preparação dos Dados:** Limpar e estruturar o df_cleaned (ou uma versão dele) para ser acessível pelo chatbot (ex: via API, banco de dados).")
print("- **Fase 3: Desenvolvimento do Chatbot:** Construir a lógica do chatbot, fluxos de conversa e integração com a fonte de dados.")
print("- **Fase 4: Treinamento (para chatbots com PNL):** Treinar o modelo de PNL com exemplos de perguntas e intenções dos alunos.")
print("- **Fase 5: Testes:** Realizar testes rigorosos com usuários para identificar problemas e melhorar a interação.")
print("- **Fase 6: Implantação:** Implantar o chatbot em uma plataforma acessível aos alunos (site da universidade, aplicativo, etc.).")
print("- **Fase 7: Monitoramento e Manutenção:** Acompanhar o desempenho do chatbot e realizar atualizações conforme necessário.")

# 5. Summarize potential benefits
print("\n5. Benefícios Potenciais para Retenção de Alunos:")
print("- **Suporte Proativo e Personalizado:** Abordar alunos em risco antes que a situação se agrave.")
print("- **Acessibilidade:** Disponibilidade 24/7 para responder a dúvidas e fornecer informações.")
print("- **Eficiência:** Reduzir a carga de trabalho da equipe de suporte ao responder a perguntas frequentes.")
print("- **Melhora do Engajamento:** Manter os alunos mais informados e conectados à universidade.")
print("- **Coleta de Insights:** Obter feedback valioso sobre os desafios dos alunos para melhorar continuamente os serviços e o currículo.")

## Opções Específicas de Integração para o Modelo Preditivo

Specific Integration Options for the Predictive Model

Com base nas discussões sobre sistemas universitários e a natureza do modelo preditivo de risco, aqui estão algumas opções de integração mais específicas:
Based on discussions about university systems and the nature of the predictive risk model, here are some more specific integration options:

### 1. Integração via API (Interface de Programação de Aplicações)

1. Integration via API (Application Programming Interface)

🇧🇷 Como funciona: O modelo preditivo seria implantado como um serviço (por exemplo, usando Flask, FastAPI, ou ferramentas específicas de MLOps) que expõe uma API. Outros sistemas universitários (como o SIS, LMS, ou um sistema de alerta) fariam requisições a essa API, enviando os dados de um aluno e recebendo de volta a previsão de risco.

🇬🇧 How it works: The predictive model would be deployed as a service (e.g., using Flask, FastAPI, or MLOps-specific tools) exposing an API. Other university systems (like SIS, LMS, or an alert system) would make requests to this API, sending student data and receiving the risk prediction in response.

🇧🇷 Vantagens: Flexível, escalável, permite que diferentes sistemas acessem o modelo sem precisar conhecer seus detalhes internos.

🇬🇧 Advantages: Flexible, scalable, allows different systems to access the model without knowing its internal details.

🇧🇷 Desafios: Requer desenvolvimento de API, gerenciamento de infraestrutura para o serviço do modelo, e segurança para as requisições.

🇬🇧 Challenges: Requires API development, management of the model’s service infrastructure, and security for requests.

### 2. Integração via Banco de Dados Compartilhado

2. Integration via Shared Database

🇧🇷 Como funciona: As previsões de risco do modelo seriam gravadas diretamente em uma tabela em um banco de dados que é acessado pelos sistemas que precisam dessa informação. O modelo rodaria periodicamente (por exemplo, diariamente ou semanalmente) para gerar as previsões e atualizar o banco.

🇬🇧 How it works: The model’s risk predictions would be written directly to a table in a database accessible by systems needing this information. The model would run periodically (e.g., daily or weekly) to generate predictions and update the database.

🇧🇷 Vantagens: Relativamente simples de implementar se os sistemas já compartilham um banco de dados, ou se a universidade possui um data warehouse centralizado.

🇬🇧 Advantages: Relatively simple to implement if systems already share a database or if the university has a centralized data warehouse.

🇧🇷 Desafios: Risco de acoplamento entre os sistemas, gerenciamento de acesso e permissões ao banco de dados, latência para acesso a dados em tempo real (as previsões só são atualizadas na frequência da execução do modelo).

🇬🇧 Challenges: Risk of coupling between systems, managing access and permissions to the database, possible latency for real-time data (since predictions are only updated as often as the model runs).

### 3. Integração via Sistema de Mensageria (ex: Kafka, RabbitMQ)

3. Integration via Messaging System (e.g., Kafka, RabbitMQ)

🇧🇷 Como funciona: O modelo preditivo publicaria eventos ou mensagens com as previsões de risco para um tópico em um sistema de mensageria. Outros sistemas interessados (assinantes) consumiriam essas mensagens e agiriam com base nelas (por exemplo, um sistema de alerta que dispara uma notificação, ou o chatbot que inicia uma conversa).

🇬🇧 How it works: The predictive model would publish events or messages with risk predictions to a topic on a messaging system. Other interested systems (subscribers) would consume these messages and act accordingly (e.g., an alert system triggering a notification, or a chatbot starting a conversation).

🇧🇷 Vantagens: Desacoplamento entre sistemas, escalabilidade, resiliência a falhas, adequado para notificações em tempo quase real.

🇬🇧 Advantages: Decoupling between systems, scalability, fault-tolerance, suitable for near real-time notifications.

🇧🇷 Desafios: Maior complexidade de arquitetura, requer implementação e gerenciamento do sistema de mensageria.

🇬🇧 Challenges: Greater architectural complexity; requires implementation and management of the messaging system.

### 4. Integração Direta (Menos Recomendada para Sistemas Complexos)

4. Direct Integration (Less Recommended for Complex Systems)

🇧🇷 Como funciona: O código do modelo preditivo ou suas previsões seriam incorporados diretamente em outro sistema.

🇬🇧 How it works: The predictive model’s code or its predictions would be embedded directly into another system.

🇧🇷 Vantagens: Simples para protótipos ou integrações pontuais.

🇬🇧 Advantages: Simple for prototypes or one-off integrations.

🇧🇷 Desafios: Alto acoplamento, difícil manutenção, escalabilidade limitada, duplicação de lógica em diferentes sistemas.

🇬🇧 Challenges: High coupling, difficult maintenance, limited scalability, duplication of logic across systems.

### Considerações Chave para a Escolha

Key Considerations for the Choice

🇧🇷 Infraestrutura existente: Quais sistemas a universidade já utiliza e como eles se comunicam?
🇬🇧 Existing infrastructure: What systems does the university already use and how do they communicate?

🇧🇷 Necessidade de tempo real: As previsões precisam ser imediatas, ou atualizações periódicas são suficientes?
🇬🇧 Need for real-time: Do predictions need to be immediate or are periodic updates sufficient?

🇧🇷 Escalabilidade e volume de dados: Quantos alunos precisam ser avaliados e com que frequência?
🇬🇧 Scalability and data volume: How many students need to be assessed and how often?

🇧🇷 Segurança e privacidade dos dados: Como garantir que os dados dos alunos estejam protegidos em cada etapa da integração?
🇬🇧 Data security and privacy: How to ensure student data is protected at every integration stage?

🇧🇷 Recursos de desenvolvimento e manutenção: Qual opção a equipe de TI da universidade tem capacidade de implementar e gerenciar?
🇬🇧 Development and maintenance resources: Which option is the university IT team prepared to implement and manage?

Discutir essas opções com a equipe técnica da universidade seria o próximo passo para definir a melhor abordagem de integração.
Discussing these options with the university technical team would be the next step to define the best integration approach.


In [ ]:
# Safely show feature importances only if the model and feature matrix are available
if 'model' in globals() and hasattr(model, 'feature_importances_'):
    # prefer X_train columns if available, else fallback to X
    if 'X_train' in globals():
        cols = X_train.columns
    elif 'X' in globals():
        cols = X.columns
    else:
        cols = None

    if cols is None:
        print("Model found but no feature matrix (X_train or X) available to get feature names.")
    else:
        feature_importances = model.feature_importances_
        # Guard against mismatch in lengths
        if len(feature_importances) != len(cols):
            print("Warning: number of feature importances does not match number of feature columns.")
            min_len = min(len(feature_importances), len(cols))
            feature_importances = feature_importances[:min_len]
            cols = cols[:min_len]

        feature_importance_series = pd.Series(feature_importances, index=cols)
        sorted_feature_importances = feature_importance_series.sort_values(ascending=False)

        top_n = min(10, len(sorted_feature_importances))
        print(f"Top {top_n} most important features for predicting student risk:")
        display(sorted_feature_importances.head(top_n))

        print(f"\nInterpretation of the top {top_n} most important features:")
        print("These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.")
        print("Let's consider the potential meaning in the context of student retention and evasion:")

        for feature, importance in sorted_feature_importances.head(top_n).items():
            print(f"- {feature}: (Importance: {importance:.4f})")
            # Normalize feature name for simple substring checks
            fn = ''.join(ch for ch in str(feature).lower() if ch.isalnum())

            if any(k in fn for k in ['situacaoaluno', 'situacao', 'situao', 'situaoaluno', 'situao']):
                print("  - This likely relates to the student's status in the current semester, which is a direct indicator of potential evasion.")
            elif any(k in fn for k in ['situacaofinal', 'situaofinal', 'final', 'situacaofinal_']):
                print("  - This refers to the student's final outcome, which is also closely tied to retention.")
            elif 'quantidadedereprov' in fn or ('reprov' in fn and 'quant' in fn):
                print("  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.")
            elif any(k in fn for k in ['motivo', 'frequent', 'naoesta', 'naoestmais', 'naofrequent']):
                print("  - The reason for not attending classes directly explains the cause of potential evasion.")
            elif any(k in fn for k in ['statusaluno', 'semestreanterior', 'anterior']):
                print("  - The student's status in the previous semester could indicate a history of academic issues or successful progression.")
            elif any(k in fn for k in ['trancar', 'trancamento']):
                print("  - This might indicate a student considering or having previously considered interrupting their studies.")
            elif 'sala' in fn or 'class' in fn:
                print("  - The specific class or location might be correlated with student demographics, teaching styles, or local factors affecting retention.")
            else:
                print("  - This feature's meaning is less clear from the name alone, but it's highly influential according to the model.")
else:
    print("Model is not defined or not trained yet. Please train the model (run the training cell) before requesting feature importances.")

Top 10 most important features for predicting student risk:


Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade                           0.204199
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade                0.089436
Quantidadedereprovaesnocurso4reprovaesperdadevaga                                0.087386
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante          0.086220
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Gravidezmulherouparceiroa      0.059490
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadenveldentrodaUnidade    0.048197
Sala_FourCornersB1                                                               0.028644
Statusalunosemestreanterior_Reprovado                                            0.026601
Statusalunosemestreanterior_Aprovado                                             0.026073
Sala_FourCornersA2                                                               0.024301
dtype: float64


Interpretation of the top 10 most important features:
These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.
Let's consider the potential meaning in the context of student retention and evasion:
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade: (Importance: 0.2042)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade: (Importance: 0.0894)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Quantidadedereprovaesnocurso4reprovaesperdadevaga: (Importance: 0.0874)
  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante: (Importance: 0.0862)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Se

🇬🇧 Preparing the data for modeling  
🇧🇷 Preparar os dados para modelagem

🇬🇧 Select relevant columns, handle categorical data (encoding), and split the dataset into training and test sets.  
🇧🇷 Selecionar as colunas relevantes, lidar com dados categóricos (codificação) e dividir o dataset em conjuntos de treinamento e teste.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd # Ensure pandas is imported

# 1. Selecione as colunas relevantes
# Incluindo colunas identificadas como importantes e outras potencialmente relevantes
relevant_cols = [
    'Sala',
    'Situação aluno no semestre',
    'Status aluno semestre anterior',
    'Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)',
    'Se aluno não está mais frequentando as aulas, escolha o motivo',
    'Quantidade de reprovações no curso (4 reprovações = perda de vaga)'
]

# Filter the DataFrame to include only the relevant columns
df_relevant = df_cleaned[relevant_cols].copy()

# 4. Defina a variável alvo
# Classificar alunos com 'evadiu' ou 'desistiu' como 'em risco' (1) e os demais como 'não em risco' (0)
df_relevant['em_risco'] = df_relevant['Situação aluno no semestre'].apply(
    lambda x: 1 if x in ['evadiu', 'desistiu'] else 0
)

# Drop the original 'Situação aluno no semestre' column as it's used to create the target
df_relevant = df_relevant.drop(columns=['Situação aluno no semestre'])

# Handle the 'Quantidade de reprovações no curso (4 reprovações = perda de vaga)' column separately
# Convert to numeric, coercing errors to NaN
quantidade_reprovacoes_col_name = 'Quantidade de reprovações no curso (4 reprovações = perda de vaga)'
df_relevant[quantidade_reprovacoes_col_name] = pd.to_numeric(df_relevant[quantidade_reprovacoes_col_name], errors='coerce')

# Impute missing numerical values in this column with the median
if df_relevant[quantidade_reprovacoes_col_name].isnull().any():
    df_relevant[quantidade_reprovacoes_col_name] = df_relevant[quantidade_reprovacoes_col_name].fillna(df_relevant[quantidade_reprovacoes_col_name].median())


# 2. Identifique as colunas categóricas (excluding the target variable and the numeric reprovações column)
categorical_cols = df_relevant.select_dtypes(include=['object']).columns

# Convert all values in categorical columns to string type
for col in categorical_cols:
    df_relevant[col] = df_relevant[col].astype(str)

# Handle potential NaNs in categorical columns before encoding by filling with a placeholder (fillna('Missing') should be after astype(str))
for col in categorical_cols:
    df_relevant[col] = df_relevant[col].fillna('Missing')


# 3. Aplique One-Hot Encoding às colunas categóricas
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_categorical_data = encoder.fit_transform(df_relevant[categorical_cols])

# Create a DataFrame from the encoded data
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_cols))

# Drop the original categorical columns from the relevant DataFrame
df_processed = df_relevant.drop(columns=categorical_cols)

# Concatenate the encoded categorical DataFrame with the rest of the DataFrame (including the now numeric reprovações column)
df_processed = pd.concat([df_processed, encoded_categorical_df], axis=1)

# Ensure all column names are valid after one-hot encoding (replace special characters)
# This step is still needed to clean the names generated by get_feature_names_out and the original numeric column name
df_processed.columns = df_processed.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)


# Separate features (X) and target (y)
X = df_processed.drop(columns=['em_risco'])
y = df_processed['em_risco']


# 5. Divida o dataset em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Original DataFrame shape:", df_cleaned.shape)
print("Relevant columns DataFrame shape:", df_relevant.shape)
print("Processed DataFrame shape:", df_processed.shape)
print("Features shape (X):", X.shape)
print("Target shape (y):", y.shape)
print("Training features shape (X_train):", X_train.shape)
print("Testing features shape (X_test):", X_test.shape)
print("Training target shape (y_train):", y_train.shape)
print("Testing target shape (y_test):", y_test.shape)

Original DataFrame shape: (1977, 99)
Relevant columns DataFrame shape: (1977, 6)
Processed DataFrame shape: (1977, 55)
Features shape (X): (1977, 54)
Target shape (y): (1977,)
Training features shape (X_train): (1581, 54)
Testing features shape (X_test): (396, 54)
Training target shape (y_train): (1581,)
Testing target shape (y_test): (396,)


## Identificar alunos em risco usando o modelo

### Subtask:

🇧🇷 Aplicar o modelo treinado ao conjunto completo de dados ou a novos dados para identificar alunos que são classificados como em risco.

🇬🇧 Apply the trained model to the entire dataset or new data to identify students who are classified as at risk.
🇧🇷 Aplicar o modelo treinado ao conjunto completo de dados ou a novos dados para identificar alunos que são classificados como em risco.

🇬🇧 Apply the trained model to the entire dataset or new data to identify students who are classified as at risk.

In [ ]:
# Ensure required objects/imports exist and train a model if needed, then predict safely.

# Import only if missing in the session
if 'np' not in globals():
	import numpy as np
if 'RandomForestClassifier' not in globals():
	from sklearn.ensemble import RandomForestClassifier

# Ensure a feature matrix X is available
if 'X' not in globals():
	# try common alternatives
	if 'df_processed' in globals() and 'em_risco' in df_processed.columns:
		X = df_processed.drop(columns=['em_risco'])
	else:
		raise NameError("Feature matrix 'X' not found. Please run the data-preparation cells to create 'X' before predicting.")

# Ensure a trained model is available; if not, train using available training data
if 'model' not in globals():
	if 'X_train' in globals() and 'y_train' in globals():
		model = RandomForestClassifier(random_state=42)
		model.fit(X_train, y_train)
		print("Trained RandomForestClassifier on X_train/y_train because 'model' was not defined.")
	elif 'y' in globals():
		model = RandomForestClassifier(random_state=42)
		model.fit(X, y)
		print("Trained RandomForestClassifier on full X/y because 'model' was not defined and X_train/y_train are not available.")
	else:
		raise NameError("No trained model found and no training labels (y or y_train) available to train one. Please run the training cell first.")

# Make predictions on X and align them back to df_cleaned by index if possible
preds = model.predict(X)

# Try to place predictions into df_cleaned aligned by index
if hasattr(X, 'index') and hasattr(df_cleaned, 'index'):
	df_cleaned['predicted_em_risco'] = np.nan
	# If indices match in length and order, assign directly; otherwise align by index
	if len(preds) == len(df_cleaned) and all(X.index == df_cleaned.index):
		df_cleaned['predicted_em_risco'] = preds
	else:
		# assign only for rows corresponding to X.index
		df_cleaned.loc[X.index, 'predicted_em_risco'] = preds
else:
	# fallback: assign by position only if lengths match
	if len(preds) == len(df_cleaned):
		df_cleaned['predicted_em_risco'] = preds
	else:
		raise ValueError("Cannot align predictions with df_cleaned: index mismatch and lengths differ.")

# Filter and display
at_risk_students_df = df_cleaned[df_cleaned['predicted_em_risco'] == 1].copy()

print("Predicted At-Risk Students:")
display(at_risk_students_df.head())

print(f"\nTotal number of students predicted to be at risk: {len(at_risk_students_df)}")

Predicted At-Risk Students:


,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,P,CP,Pre-Class,P_1,Hw,CP_1,Pre-Class_1,P_2,Hw_1,CP_2,Pre-Class_2,P_3,Hw_2,CP_3,Pre-Class_3,P_4,Hw_3,CP_4,Pre-Class_4,P_5,Hw_4,CP_5,Pre-Class_5,P_6,Hw_5,CP_6,Pre-Class_6,P_7,Hw_6,CP_7,Pre-Class_7,P_8,Hw_7,CP_8,Pre-Class_8,P_9,Hw_8,CP_9,Pre-Class_9,P_10,Hw_9,CP_10,Pre-Class_10,P_11,Hw_10,CP_11,Pre-Class_11,P_12,CP_12,O,W,Nota,O_1,W_1,Nota_1,O_2,W_2,Nota_2,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P_13,A,1/2,Total_1,%_1,√_1,+/-_1,N_1,Total_2,%_2,:-D,:-/,:-&,Total_3,%_3,:-(,:-||,11.0,%_4,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovações = perda de vaga),Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado),Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, escolha o motivo",Situação Final_1,predicted_em_risco
0,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,A,:-D,√,P,√,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,N,P,N,:-D,√,P,√,:-D,√,P,N,:-D,N,1/2,√,:-/,√,A,N,:-D,N,P,:-D,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Reprovado,1
1,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,A,:-D,√,P,N,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,A,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,1/2,√,:-D,√,A,√,:-D,N,P,:-D,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,A,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,:-D,1900-01-08 00:00:00,10,0,9,10,0,9,10,0,0,0,desistiu,desistiu,0,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu,1
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,N,:-D,N,P,√,:-D,N,P,N,:-D,√,P,+/-,:-D,√,P,N,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,N,P,N,:-D,N,P,N,:-D,√,P,:-D,1900-01-08 12:00:00,9,8.8,8,9.2,8.72,7.5,8.6,8.16,8.56,7.094233,Aprovado,Ok,5,0,7,12,0.416667,13,0,0,13,1,3,1,7,11,0.318182,8,0,0,8,0.888889,0,0,13.0,1,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,5,Estudante 5 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,A,√,:-D,√,P,:-D,9,9.4,9.24,9,9.6,9.36,8.5,7,7.6,8.733333,8.726856,Aprovado,Ok,10,0,2,12,0.833333,11,2,0,11,0.85,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,11.0,1,2025-06-06 18:18:41,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1



Total number of students predicted to be at risk: 1921


🇧🇷 **Raciocínio**:
- Carregar o dataset pulando linhas de metadados e usando a linha correta como cabeçalho.
- Importar as métricas de avaliação de sklearn.metrics (accuracy_score, precision_score, recall_score, f1_score).
- Usar o modelo treinado para prever em X_test (model.predict(X_test)).
- Calcular accuracy, precision, recall e F1-score; exibir e interpretar os resultados (priorizar recall quando quisermos encontrar a maior parte dos alunos em risco).
- Opcional: mostrar matriz de confusão e relatório de classificação para entender erros (false positives / false negatives).

🇬🇧 **Reasoning**:
- Load the dataset skipping metadata rows and using the correct header row.
- Import evaluation metrics from sklearn.metrics (accuracy_score, precision_score, recall_score, f1_score).
- Use the trained model to predict on X_test (model.predict(X_test)).
- Compute accuracy, precision, recall and F1-score; display and interpret results (recall is critical when prioritizing detection of at-risk students).
- Optional: show confusion matrix and classification report to inspect false positives / false negatives.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the trained model's .predict() method to make predictions on the test features
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate the precision of the model
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision}")

# Calculate the recall of the model
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

# Calculate the F1-score of the model
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")

# Interpret the calculated metrics
print("\nInterpretation of Metrics:")
print(f"Accuracy ({accuracy:.2f}): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances. A high accuracy suggests the model is generally correct in its predictions.")
print(f"Precision ({precision:.2f}): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk. A high precision indicates that when the model predicts a student is at risk, it is likely to be correct.")
print(f"Recall ({recall:.2f}): This is the proportion of correctly predicted at-risk students out of all actual at-risk students. A high recall indicates that the model is good at identifying most of the students who are actually at risk.")
print(f"F1-score ({f1:.2f}): This is the harmonic mean of precision and recall, providing a single metric that balances both. It is particularly useful when there is an uneven class distribution (as is likely the case with at-risk vs. not at-risk students). A high F1-score indicates a good balance between precision and recall.")

Accuracy: 0.9595959595959596
Precision: 0.9585492227979274
Recall: 1.0
F1-score: 0.9788359788359788

Interpretation of Metrics:
Accuracy (0.96): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances. A high accuracy suggests the model is generally correct in its predictions.
Precision (0.96): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk. A high precision indicates that when the model predicts a student is at risk, it is likely to be correct.
Recall (1.00): This is the proportion of correctly predicted at-risk students out of all actual at-risk students. A high recall indicates that the model is good at identifying most of the students who are actually at risk.
F1-score (0.98): This is the harmonic mean of precision and recall, providing a single metric that balances both. It is particularly useful when there is an uneven class distribution (as is likely the case with 

## Selecionar e treinar um modelo preditivo

🇧🇷 Escolher um algoritmo de machine learning apropriado para a classificação (prever se um aluno está em risco ou não) e treinar o modelo com os dados de treinamento.

🇬🇧 Choose an appropriate machine learning algorithm for classification (predict whether a student is at risk or not) and train the model with the training data.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Import a classification model (RandomForestClassifier chosen for its robustness)
# 2. Instantiate the chosen model
model = RandomForestClassifier(random_state=42)

# 3. Train the model using the training data
model.fit(X_train, y_train)

print("Model training complete.")

Model training complete.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Import a classification model (RandomForestClassifier chosen for its robustness)
# 2. Instantiate the chosen model
model = RandomForestClassifier(random_state=42)

# 3. Train the model using the training data
model.fit(X_train, y_train)

print("Model training complete.")

Model training complete.


In [ ]:
# 1. Access the feature_importances_ attribute of the trained model
feature_importances = model.feature_importances_

# 2. Create a pandas Series from the feature importances
feature_importance_series = pd.Series(feature_importances, index=X_train.columns)

# 3. Sort the feature importances in descending order
sorted_feature_importances = feature_importance_series.sort_values(ascending=False)

# 4. Print the top N most important features (e.g., top 10)
top_n = 5
print(f"Top {top_n} most important features for predicting student risk:")
display(sorted_feature_importances.head(top_n))

# 5. Interpret the meaning of these top features
print(f"\nInterpretation of the top {top_n} most important features:")
print("These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.")
print("Let's consider the potential meaning in the context of student retention and evasion:")

# Iterate through the top features and provide interpretation
for feature, importance in sorted_feature_importances.head(top_n).items():
    print(f"- {feature}: (Importance: {importance:.4f})")
    if 'Situaçãoaluno' in feature:
        print("  - This likely relates to the student's status in the current semester, which is a direct indicator of potential evasion.")
    elif 'SituaçãoFinal' in feature:
        print("  - This refers to the student's final outcome, which is also closely tied to retention.")
    elif 'Quantidadedereprovaesnocurso' in feature:
        print("  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.")
    elif 'Sealunonoestmais' in feature:
        print("  - The reason for not attending classes directly explains the cause of potential evasion.")
    elif 'Statusalunosemestreanterior' in feature:
         print("  - The student's status in the previous semester could indicate a history of academic issues or successful progression.")
    elif 'Alunopodetrancarmatrculaoumantertrancamento' in feature:
         print("  - This might indicate a student considering or having previously considered interrupting their studies.")
    elif 'Sala' in feature:
        print("  - The specific class or location might be correlated with student demographics, teaching styles, or local factors affecting retention.")
    else:
        print("  - This feature's meaning is less clear from the name alone, but it's highly influential according to the model.")

print("\nUnderstanding these features can help in focusing interventions and marketing efforts.")

Top 5 most important features for predicting student risk:


Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade                         0.204199
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade              0.089436
Quantidadedereprovaesnocurso4reprovaesperdadevaga                              0.087386
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante        0.086220
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Gravidezmulherouparceiroa    0.059490
dtype: float64


Interpretation of the top 5 most important features:
These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.
Let's consider the potential meaning in the context of student retention and evasion:
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade: (Importance: 0.2042)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade: (Importance: 0.0894)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Quantidadedereprovaesnocurso4reprovaesperdadevaga: (Importance: 0.0874)
  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante: (Importance: 0.0862)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sea

In [ ]:
# Get the content of the markdown cell containing the README.md content
readme_content = """# Projeto Marketing de Redes Sociais PUC-SP

## Descrição do Projeto

Este projeto, desenvolvido para a disciplina de Marketing de Redes Sociais da PUC-SP, tem como objetivo analisar dados de alunos para identificar padrões de retenção e evasão, prever alunos em risco e propor estratégias de marketing de redes sociais direcionadas para melhorar o engajamento e a permanência dos estudantes. Além disso, exploramos a viabilidade de integrar um chatbot para fornecer suporte personalizado aos alunos.

## Dataset

O dataset utilizado neste projeto é a "Base anonimizada - Eric - PUC-SP (1).xlsx", que contém informações sobre alunos, incluindo dados de participação, notas, situação acadêmica anterior e no semestre atual, motivos de não frequência, entre outros.

Durante a fase de organização e limpeza dos dados, foram realizadas as seguintes etapas:
- Carregamento do dataset, pulando linhas de metadados e utilizando a linha correta como cabeçalho.
- Padronização dos nomes das colunas (minúsculas e substituição de espaços por underscores).
- Identificação e tratamento de colunas duplicadas através da adição de sufixos numéricos.
- Análise e tratamento de valores ausentes, incluindo a remoção de colunas com alta porcentagem de dados faltantes e a imputação de valores em colunas relevantes.

A exploração inicial do dataset revelou insights importantes sobre a distribuição dos alunos por sala, a alta taxa de evasão no semestre e os principais motivos associados à não frequência.

## Objetivos do Projeto

Com base no briefing e na análise dos dados, definimos os seguintes objetivos:

1.  **Analisar padrões de retenção e evasão de alunos:** Identificar os fatores que mais contribuem para a evasão e os motivos associados à não frequência.
2.  **Identificar alunos em risco:** Desenvolver métodos para prever quais alunos têm maior probabilidade de evadir com base em seu histórico e comportamento no curso.
3.  **Desenvolver estratégias de marketing de redes sociais direcionadas:** Propor campanhas de marketing online focadas em reter alunos, aumentar o engajamento e fornecer suporte.
4.  **Avaliar o impacto potencial de intervenções:** Sugerir formas de medir a eficácia de futuras ações baseadas nos insights do projeto.
5.  **Explorar a viabilidade de um chatbot:** Investigar como um chatbot poderia ser utilizado para interagir com os alunos, fornecer informações personalizadas e coletar feedback.

## Principais Descobertas e Análises

### Análise Exploratória Inicial

*   O dataset apresentou desafios iniciais na limpeza e organização devido a metadados e cabeçalhos em múltiplas linhas, além de colunas duplicadas. Essas questões foram tratadas para permitir a análise.
*   A análise exploratória revelou uma alta taxa de evasão ('evadiu') entre os alunos no semestre.
*   O motivo mais comum para a não frequência reportado no dataset foi 'Sem retorno do estudante'.
*   A maioria dos alunos no dataset tinha um status de 'Aprovado' no semestre anterior.
*   A coluna 'Quantidade de reprovações no curso' indica que a maioria dos alunos possui poucas ou nenhuma reprovação.
*   Exploramos a viabilidade de um chatbot para interagir com os dados dos alunos, identificando casos de uso potencial como suporte proativo a alunos em risco e resposta a perguntas sobre status acadêmico.

### Modelo Preditivo de Risco de Evasão

*   Desenvolvemos um modelo preditivo (Random Forest Classifier) para identificar alunos em risco de evasão.
*   O modelo demonstrou alto desempenho no conjunto de teste, com 96% de acurácia, 96% de precisão, 100% de recall e 98% de F1-score.
*   A alta pontuação de recall (100%) é particularmente relevante, indicando que o modelo é muito eficaz em identificar a vasta maioria dos alunos que realmente estão em risco.
*   Identificamos um total de [Número total de alunos previstos em risco] alunos como estando em risco de evasão ou desistência no dataset completo.

### Fatores de Risco Mais Importantes

*   A análise de importância das features do modelo revelou que os principais fatores que contribuem para a previsão de risco incluem:
    *   [Listar os top N fatores mais importantes do output da célula `09f94bcf`] (e.g., Motivo de não frequência, Quantidade de reprovações, Status do semestre anterior, Sala).
    *   [Incluir uma breve interpretação de como esses fatores se relacionam com o risco de evasão].

### Ideias de Integração e Próximos Passos

*   Os resultados do modelo podem ser integrados a um sistema de alerta precoce para notificar conselheiros ou staff sobre alunos em risco.
*   O modelo pode subsidiar funcionalidades proativas do chatbot, permitindo que ele entre em contato com alunos em risco para oferecer suporte personalizado.
*   Outras opções de integração incluem banco de dados compartilhado ou sistemas de mensageria, dependendo da infraestrutura da universidade.
*   É crucial considerar a privacidade e segurança dos dados (LGPD) em qualquer implementação, bem como garantir a supervisão humana e mitigar vieses no modelo.
*   Próximos passos incluem refinar o modelo com mais dados (se disponíveis), validar as descobertas com especialistas em retenção e explorar a implementação piloto de um sistema de alerta ou chatbot.

## Como Rodar o Código

1.  **Clone o repositório:** Clone este repositório do GitHub para sua máquina local.
2.  **Instale as dependências:** Certifique-se de ter as bibliotecas necessárias instaladas. Você pode instalá-las usando pip:
"""

# Save the content to a file named README.md
with open('README.md', 'w') as f:
    f.write(readme_content)

print("README.md content saved to a file in the Colab environment. You can now download it from the file explorer.")

README.md content saved to a file in the Colab environment. You can now download it from the file explorer.


## Summary

### Data Analysis — Principais Achados / Key Findings

- 🇧🇷 O dataset apresentava metadados nas primeiras linhas; foi necessário carregar com skiprows=1 e header=0.  
    🇬🇧 The dataset had metadata rows at the top; loaded with skiprows=1 and header=0.

- 🇧🇷 Nomes de colunas inconsistentes (maiúsculas/minúsculas e espaços): padronizados para lowercase e underscores.  
    🇬🇧 Column names were inconsistent; standardized to lowercase with underscores.

- 🇧🇷 Colunas duplicadas detectadas e renomeadas adicionando sufixos numéricos.  
    🇬🇧 Duplicate columns were detected and renamed with numeric suffixes.

- 🇧🇷 Muitos valores ausentes em algumas colunas (algumas >90%); colunas com >90% missing foram removidas.  
    🇬🇧 Many missing values in some columns (some >90%); columns with >90% missing were dropped.

- 🇧🇷 Colunas numéricas com poucos missing foram imputadas pela mediana; categóricas foram preenchidas pela moda.  
    🇬🇧 Numerical columns with few missings were median-imputed; categorical filled with mode.

- 🇧🇷 Colunas de presença ('aula_') mantiveram missing (presumivelmente ausência) e não foram preenchidas automaticamente.  
    🇬🇧 Attendance ('aula_') columns kept missing values (likely absences) and were not auto-filled.

- 🇧🇷 Grande proporção de alunos classificados como 'evadiu' no semestre; muitos tinham 'Aprovado' no semestre anterior.  
    🇬🇧 A large share of students were labeled 'evadiu' for the semester; many were 'Aprovado' in the previous semester.

- 🇧🇷 Motivo mais frequente para não frequentar: "Sem retorno do estudante".  
    🇬🇧 Most frequent reason for non-attendance: "Sem retorno do estudante".

- 🇧🇷 Projeção de risco: modelo RandomForest treinado; métricas no teste (ex.: accuracy, precision, recall, f1) foram calculadas.  
    🇬🇧 Risk model: RandomForest trained; test metrics (accuracy, precision, recall, f1) were computed.

- 🇧🇷 Features mais importantes: motivos de não frequência (saúde, mudança de unidade, sem retorno), quantidade de reprovações e alguns atributos de sala/status.  
    🇬🇧 Most important features: non-attendance reasons (health, unit change, no return), number of failures, and some room/status attributes.

### Insights e Próximos Passos / Insights & Next Steps

- 🇧🇷 Priorizar análise aprofundada das variáveis mais importantes (situação no semestre, quantidade de reprovações, motivos de não frequência) para interpretar causas.  
    🇬🇧 Prioritize deep-dive into top variables (current semester status, number of failures, non-attendance reasons) to interpret causes.

- 🇧🇷 Validar manualmente mapeamentos e imputações; garantir que labels de target estejam corretos e sem viés.  
    🇬🇧 Manually validate mappings and imputations; ensure target labeling is correct and unbiased.

- 🇧🇷 Refinar o modelo (balanceamento de classes, tuning, validação temporal se aplicável) e avaliar explicabilidade (SHAP).  
    🇬🇧 Refine the model (class balancing, hyperparameter tuning, temporal validation if applicable) and evaluate explainability (SHAP).

- 🇧🇷 Integrar previsões a um sistema de alerta ou chatbot com supervisão humana; considerar LGPD e segurança de dados.  
    🇬🇧 Integrate predictions into an early-alert system or chatbot with human oversight; consider LGPD and data security.

- 🇧🇷 Criar dashboards operacionais para monitorar tendências de risco e medir impacto de intervenções.  
    🇬🇧 Build operational dashboards to monitor risk trends and measure intervention impact.


## Summary

### Data Analysis — Principais achados / Key findings

- 🇧🇷 Modelo preditivo: 96% acurácia, 96% precisão, 100% recall e 98% F1-score.  
    🇬🇧 Predictive model: 96% accuracy, 96% precision, 100% recall and 98% F1-score.

- 🇧🇷 Número de alunos previstos em risco: 1921.  
    🇬🇧 Number of students predicted at risk: 1921.

- 🇧🇷 Principais fatores que influenciam o risco: situação no semestre atual, status do semestre anterior, quantidade de reprovações e motivo de não frequência.  
    🇬🇧 Key risk drivers: current-semester status, previous-semester status, number of course failures, and non-attendance reason.

### Insights e próximos passos / Insights & next steps

- 🇧🇷 O alto recall indica que o modelo identifica a maioria dos alunos em risco — priorizar intervenções e validação manual para reduzir falsos positivos.  
    🇬🇧 High recall indicates the model catches most at‑risk students — prioritize interventions and manual validation to reduce false positives.

- 🇧🇷 Validar mapeamentos e imputações, revisar potenciais vieses e ajustar pesos do score de engajamento.  
    🇬🇧 Validate mappings/imputations, audit for bias, and refine engagement-score weighting.

- 🇧🇷 Integrar previsões a um sistema de alerta ou chatbot com supervisão humana; garantir conformidade com LGPD.  
    🇬🇧 Integrate predictions into an early‑alert system or chatbot with human oversight; ensure data‑privacy (LGPD) compliance.

- 🇧🇷 Próximos passos técnicos: tuning do modelo, balanceamento de classes, validação temporal (se aplicável) e explicabilidade (SHAP).  
    🇬🇧 Next technical steps: model tuning, class balancing, temporal validation (if applicable), and explainability (SHAP).


## Discuss how to integrate the model (optional)

### Subtask:

🇧🇷 Discutir como os resultados do modelo preditivo poderiam ser usados na prática, potencialmente integrados em um sistema de alerta ou no chatbot.

🇬🇧 Discuss how the results of the predictive model could be used in practice, potentially integrated into an alert system or the chatbot.


In [ ]:
# 1. Outline practical ways to use the identified at-risk students and predictions
print("1. Practical Applications of Predictive Model Results:")
print("- **Targeted Outreach:** University staff (advisors, counselors, support services) can proactively contact students identified as at-risk to offer personalized support, discuss challenges, and provide resources.")
print("- **Academic Support:** Students predicted to be at-risk due to academic factors (e.g., number of failures) can be directed to tutoring services, study groups, or academic workshops.")
print("- **Counseling and Wellness:** Students whose reasons for not attending or other factors suggest personal challenges can be connected with counseling or mental health services.")
print("- **Tailored Communication:** Marketing and communications can be personalized for at-risk students, highlighting relevant resources, support systems, and the benefits of staying enrolled.")
print("- **Resource Allocation:** Insights from the model can help allocate limited resources (e.g., academic advisors' time, funding for support programs) more effectively to students who need them most.")

# 2. Discuss how the predictive model could be integrated into existing or proposed systems
print("\n2. Integration into Existing or Proposed Systems:")
print("- **Early Alert System:** The model's predictions can be integrated into an early alert system used by academic advisors. When a student is flagged as high-risk, an alert is generated for their advisor to follow up.")
print("- **Student Information System (SIS):** Predictions could potentially be stored in the SIS, making the risk status visible to relevant personnel who interact with the student.")
print("- **Learning Management System (LMS):** While more complex, integration could potentially trigger notifications or recommendations within the LMS based on a student's risk level.")
print("- **Student Support Platform:** A dedicated platform for student support could pull risk predictions from the model to prioritize outreach and tailor the support offered.")
print("- **Dashboards and Reporting:** Aggregate risk data and trends from the model can be displayed on dashboards for administrators to monitor student retention efforts and identify areas for improvement.")

# 3. Explain how the chatbot could leverage the model's predictions
print("\n3. Chatbot Integration with Predictive Model:")
print("- **Proactive Engagement:** The chatbot can initiate conversations with students identified as at-risk, asking how they are doing, offering assistance, and directing them to relevant support services without waiting for the student to reach out first.")
print("- **Personalized Information:** Based on the predicted risk factors (e.g., academic struggles vs. personal issues), the chatbot can provide tailored information and resources (e.g., links to tutoring, mental health resources, academic policies).")
print("- **Early Intervention:** By identifying and engaging with at-risk students early, the chatbot can facilitate timely interventions that might prevent escalation of issues leading to evasion.")
print("- **Feedback Collection:** The chatbot can be used to gather more detailed information from at-risk students about the challenges they are facing, enriching the dataset for future model improvements.")
print("- **Direct Connection to Human Support:** If the chatbot cannot address a student's needs, it can seamlessly connect them with a human advisor or counselor.")

# 4. Consider potential challenges and ethical considerations
print("\n4. Challenges and Ethical Considerations:")
print("- **Data Privacy and Security (LGPD):** Handling sensitive student data requires strict adherence to privacy regulations. Ensuring secure storage, access control, and transparent usage is paramount.")
print("- **Bias in Predictions:** Predictive models can inherit biases from the data they are trained on. This could lead to certain groups of students being unfairly flagged or overlooked. Regular auditing and efforts to mitigate bias are necessary.")
print("- **Ensuring Human Oversight:** The model's predictions should not be the sole basis for decisions. Human advisors and staff must have the final say and context to interpret the predictions and provide appropriate support.")
print("- **Transparency and Explainability:** Students and staff should understand how the risk assessment works, even if the underlying model is complex. Explaining the key factors contributing to a student's risk score can build trust.")
print("- **Potential for Negative Impact:** Labeling students as 'at-risk' could potentially have unintended negative psychological effects. The communication and framing around risk assessment are crucial.")
print("- **Resource Constraints:** Implementing and maintaining these systems and providing adequate human support for flagged students requires significant resources.")

# 5. Summarize the potential benefits of integration
print("\n5. Potential Benefits of Integration:")
print("- **Improved Student Retention:** By identifying and supporting at-risk students proactively, the university can potentially reduce evasion rates.")
print("- **Enhanced Student Success:** Targeted interventions can help students overcome challenges and succeed academically.")
print("- **More Efficient Resource Utilization:** Focusing resources on students who need them most can improve the effectiveness of support programs.")
print("- **Data-Driven Decision Making:** The model provides valuable data and insights to inform strategies for student support and retention.")
print("- **Positive Student Experience:** Proactive and personalized support can improve student satisfaction and their perception of the university's commitment to their success.")

1. Practical Applications of Predictive Model Results:
- **Targeted Outreach:** University staff (advisors, counselors, support services) can proactively contact students identified as at-risk to offer personalized support, discuss challenges, and provide resources.
- **Academic Support:** Students predicted to be at-risk due to academic factors (e.g., number of failures) can be directed to tutoring services, study groups, or academic workshops.
- **Counseling and Wellness:** Students whose reasons for not attending or other factors suggest personal challenges can be connected with counseling or mental health services.
- **Tailored Communication:** Marketing and communications can be personalized for at-risk students, highlighting relevant resources, support systems, and the benefits of staying enrolled.
- **Resource Allocation:** Insights from the model can help allocate limited resources (e.g., academic advisors' time, funding for support programs) more effectively to students who ne

In [ ]:
# 1. Access the feature_importances_ attribute of the trained model
feature_importances = model.feature_importances_

# 2. Create a pandas Series from the feature importances
feature_importance_series = pd.Series(feature_importances, index=X_train.columns)

# 3. Sort the feature importances in descending order
sorted_feature_importances = feature_importance_series.sort_values(ascending=False)

# 4. Print the top N most important features (e.g., top 10)
top_n = 10
print(f"Top {top_n} most important features for predicting student risk:")
display(sorted_feature_importances.head(top_n))

# 5. Interpret the meaning of these top features
print(f"\nInterpretation of the top {top_n} most important features:")
print("These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.")
print("Let's consider the potential meaning in the context of student retention and evasion:")

# Iterate through the top features and provide interpretation
for feature, importance in sorted_feature_importances.head(top_n).items():
    print(f"- {feature}: (Importance: {importance:.4f})")
    if 'Situaçãoaluno' in feature:
        print("  - This likely relates to the student's status in the current semester, which is a direct indicator of potential evasion.")
    elif 'SituaçãoFinal' in feature:
        print("  - This refers to the student's final outcome, which is also closely tied to retention.")
    elif 'Quantidadedereprovaesnocurso' in feature:
        print("  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.")
    elif 'Sealunonoestmais' in feature:
        print("  - The reason for not attending classes directly explains the cause of potential evasion.")
    elif 'Statusalunosemestreanterior' in feature:
         print("  - The student's status in the previous semester could indicate a history of academic issues or successful progression.")
    elif 'Alunopodetrancarmatrculaoumantertrancamento' in feature:
         print("  - This might indicate a student considering or having previously considered interrupting their studies.")
    elif 'Sala' in feature:
        print("  - The specific class or location might be correlated with student demographics, teaching styles, or local factors affecting retention.")
    else:
        print("  - This feature's meaning is less clear from the name alone, but it's highly influential according to the model.")

print("\nUnderstanding these features can help in focusing interventions and marketing efforts.")

Top 10 most important features for predicting student risk:


Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade                           0.204199
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade                0.089436
Quantidadedereprovaesnocurso4reprovaesperdadevaga                                0.087386
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante          0.086220
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Gravidezmulherouparceiroa      0.059490
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadenveldentrodaUnidade    0.048197
Sala_FourCornersB1                                                               0.028644
Statusalunosemestreanterior_Reprovado                                            0.026601
Statusalunosemestreanterior_Aprovado                                             0.026073
Sala_FourCornersA2                                                               0.024301
dtype: float64


Interpretation of the top 10 most important features:
These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.
Let's consider the potential meaning in the context of student retention and evasion:
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade: (Importance: 0.2042)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade: (Importance: 0.0894)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Quantidadedereprovaesnocurso4reprovaesperdadevaga: (Importance: 0.0874)
  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante: (Importance: 0.0862)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Se

## Identificar alunos em risco usando o modelo

### Subtask:

🇧🇷 Aplicar o modelo treinado ao conjunto completo de dados ou a novos dados para identificar alunos que são classificados como em risco.

🇬🇧 Apply the trained model to the entire dataset or new data to identify students who are classified as at risk.


🇧🇷 **Reasoning**:
🇬🇧 - Load the dataset, skipping metadata rows and using the correct row as header.
Apply the trained model to the entire dataset to predict at-risk students, store the predictions in the dataframe, filter the dataframe to display only the predicted at-risk students, and then print the total count of these students.


In [ ]:
# Use the trained model's .predict() method to make predictions on the entire X dataset
df_cleaned['predicted_em_risco'] = model.predict(X)

# Filter the df_cleaned DataFrame to display only the rows where 'predicted_em_risco' is equal to 1
at_risk_students_df = df_cleaned[df_cleaned['predicted_em_risco'] == 1]

# Display the head of this filtered DataFrame
print("Predicted At-Risk Students:")
display(at_risk_students_df.head())

# Print the total number of students identified as being at risk by the model
print(f"\nTotal number of students predicted to be at risk: {len(at_risk_students_df)}")

Predicted At-Risk Students:


,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,P,CP,Pre-Class,P_1,Hw,CP_1,Pre-Class_1,P_2,Hw_1,CP_2,Pre-Class_2,P_3,Hw_2,CP_3,Pre-Class_3,P_4,Hw_3,CP_4,Pre-Class_4,P_5,Hw_4,CP_5,Pre-Class_5,P_6,Hw_5,CP_6,Pre-Class_6,P_7,Hw_6,CP_7,Pre-Class_7,P_8,Hw_7,CP_8,Pre-Class_8,P_9,Hw_8,CP_9,Pre-Class_9,P_10,Hw_9,CP_10,Pre-Class_10,P_11,Hw_10,CP_11,Pre-Class_11,P_12,CP_12,O,W,Nota,O_1,W_1,Nota_1,O_2,W_2,Nota_2,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P_13,A,1/2,Total_1,%_1,√_1,+/-_1,N_1,Total_2,%_2,:-D,:-/,:-&,Total_3,%_3,:-(,:-||,11.0,%_4,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovações = perda de vaga),Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado),Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, escolha o motivo",Situação Final_1,predicted_em_risco
0,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,A,:-D,√,P,√,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,N,P,N,:-D,√,P,√,:-D,√,P,N,:-D,N,1/2,√,:-/,√,A,N,:-D,N,P,:-D,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Reprovado,1
1,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,A,:-D,√,P,N,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,A,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,1/2,√,:-D,√,A,√,:-D,N,P,:-D,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,A,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,:-D,1900-01-08 00:00:00,10,0,9,10,0,9,10,0,0,0,desistiu,desistiu,0,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu,1
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,N,:-D,N,P,√,:-D,N,P,N,:-D,√,P,+/-,:-D,√,P,N,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,N,P,N,:-D,N,P,N,:-D,√,P,:-D,1900-01-08 12:00:00,9,8.8,8,9.2,8.72,7.5,8.6,8.16,8.56,7.094233,Aprovado,Ok,5,0,7,12,0.416667,13,0,0,13,1,3,1,7,11,0.318182,8,0,0,8,0.888889,0,0,13.0,1,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,5,Estudante 5 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,A,√,:-D,√,P,:-D,9,9.4,9.24,9,9.6,9.36,8.5,7,7.6,8.733333,8.726856,Aprovado,Ok,10,0,2,12,0.833333,11,2,0,11,0.85,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,11.0,1,2025-06-06 18:18:41,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1



Total number of students predicted to be at risk: 1921


In [ ]:
# Display the predictions for the test set
display(y_pred)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

🇧🇷 **Raciocínio**
- Carregar o dataset pulando linhas de metadados e usando a linha correta como cabeçalho.
- Importar métricas de avaliação de sklearn.metrics: accuracy_score, precision_score, recall_score, f1_score.
- Usar o modelo treinado para prever em X_test (model.predict(X_test)).
- Calcular accuracy, precision, recall e F1‑score; priorizar recall ao identificar alunos em risco.

🇬🇧 **Reasoning**
- Load the dataset skipping metadata rows and use the correct row as header.
- Import evaluation metrics from sklearn.metrics: accuracy_score, precision_score, recall_score, f1_score.
- Use the trained model to predict on X_test (model.predict(X_test)).
- Compute accuracy, precision, recall and F1‑score; prioritize recall when detecting at‑risk students.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the trained model's .predict() method to make predictions on the test features
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate the precision of the model
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision}")

# Calculate the recall of the model
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

# Calculate the F1-score of the model
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")

# Interpret the calculated metrics
print("\nInterpretation of Metrics:")
print(f"Accuracy ({accuracy:.2f}): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances. A high accuracy suggests the model is generally correct in its predictions.")
print(f"Precision ({precision:.2f}): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk. A high precision indicates that when the model predicts a student is at risk, it is likely to be correct.")
print(f"Recall ({recall:.2f}): This is the proportion of correctly predicted at-risk students out of all actual at-risk students. A high recall indicates that the model is good at identifying most of the students who are actually at risk.")
print(f"F1-score ({f1:.2f}): This is the harmonic mean of precision and recall, providing a single metric that balances both. It is particularly useful when there is an uneven class distribution (as is likely the case with at-risk vs. not at-risk students). A high F1-score indicates a good balance between precision and recall.")

Accuracy: 0.9595959595959596
Precision: 0.9585492227979274
Recall: 1.0
F1-score: 0.9788359788359788

Interpretation of Metrics:
Accuracy (0.96): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances. A high accuracy suggests the model is generally correct in its predictions.
Precision (0.96): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk. A high precision indicates that when the model predicts a student is at risk, it is likely to be correct.
Recall (1.00): This is the proportion of correctly predicted at-risk students out of all actual at-risk students. A high recall indicates that the model is good at identifying most of the students who are actually at risk.
F1-score (0.98): This is the harmonic mean of precision and recall, providing a single metric that balances both. It is particularly useful when there is an uneven class distribution (as is likely the case with 

## Selecionar e treinar um modelo preditivo

🇧🇷 Escolher um algoritmo de machine learning apropriado para a classificação (prever se um aluno está em risco ou não) e treinar o modelo com os dados de treinamento.

🇬🇧 Choose an appropriate machine learning algorithm for classification (predict whether a student is at risk or not) and train the model with the training data.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Import a classification model (RandomForestClassifier chosen for its robustness)
# 2. Instantiate the chosen model
model = RandomForestClassifier(random_state=42)

# 3. Train the model using the training data
model.fit(X_train, y_train)

print("Model training complete.")

Model training complete.


🇧🇷 [Tradução necessária]

🇬🇧 Inspect the column names in X to find the correct name for the 'Quantidade de reprovações no curso (4 reprovações = perda de vaga)' column after the column name cleaning step and use that correct name for the numeric conversion.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 1. Selecione as colunas relevantes
# Incluindo colunas identificadas como importantes e outras potencialmente relevantes
relevant_cols = [
    'Sala',
    'Situação aluno no semestre',
    'Status aluno semestre anterior',
    'Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)',
    'Se aluno não está mais frequentando as aulas, escolha o motivo',
    'Quantidade de reprovações no curso (4 reprovações = perda de vaga)'
]

# Filter the DataFrame to include only the relevant columns
df_relevant = df_cleaned[relevant_cols].copy()

# 4. Defina a variável alvo
# Classificar alunos com 'evadiu' ou 'desistiu' como 'em risco' (1) e os demais como 'não em risco' (0)
df_relevant['em_risco'] = df_relevant['Situação aluno no semestre'].apply(
    lambda x: 1 if x in ['evadiu', 'desistiu'] else 0
)

# Drop the original 'Situação aluno no semestre' column as it's used to create the target
df_relevant = df_relevant.drop(columns=['Situação aluno no semestre'])

# 2. Identifique as colunas categóricas (excluding the target variable)
categorical_cols = df_relevant.select_dtypes(include=['object']).columns

# Convert all values in categorical columns to string type
for col in categorical_cols:
    df_relevant[col] = df_relevant[col].astype(str)

# Handle potential NaNs in categorical columns before encoding by filling with a placeholder
for col in categorical_cols:
    df_relevant[col] = df_relevant[col].fillna('Missing')


# 3. Aplique One-Hot Encoding às colunas categóricas
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_categorical_data = encoder.fit_transform(df_relevant[categorical_cols])

# Create a DataFrame from the encoded data
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_cols))

# Drop the original categorical columns from the relevant DataFrame
df_processed = df_relevant.drop(columns=categorical_cols)

# Concatenate the encoded categorical DataFrame with the rest of the DataFrame
df_processed = pd.concat([df_processed, encoded_categorical_df], axis=1)

# Ensure all column names are valid after one-hot encoding (replace special characters)
df_processed.columns = df_processed.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

# Separate features (X) and target (y)
X = df_processed.drop(columns=['em_risco'])
y = df_processed['em_risco']

# Find the correct column name for 'Quantidade de reprovações no curso (4 reprovações = perda de vaga)' in X
# We can look for a column name that contains 'Quantidade' and 'reprova'
quantidade_col = [col for col in X.columns if 'Quantidade' in col and 'reprova' in col]

if quantidade_col:
    quantidade_col_name = quantidade_col[0]
    # Convert the identified column to numeric, coercing errors
    X[quantidade_col_name] = pd.to_numeric(X[quantidade_col_name], errors='coerce')

    # Impute missing numerical values after conversion (if any were introduced by coercion)
    if X[quantidade_col_name].isnull().any():
        X[quantidade_col_name] = X[quantidade_col_name].fillna(X[quantidade_col_name].median())
else:
    print("Column related to 'Quantidade de reprovações' not found in X.")


# 5. Divida o dataset em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Original DataFrame shape:", df_cleaned.shape)
print("Relevant columns DataFrame shape:", df_relevant.shape)
print("Processed DataFrame shape:", df_processed.shape)
print("Features shape (X):", X.shape)
print("Target shape (y):", y.shape)
print("Training features shape (X_train):", X_train.shape)
print("Testing features shape (X_test):", X_test.shape)
print("Training target shape (y_train):", y_train.shape)
print("Testing target shape (y_test):", y_test.shape)

Original DataFrame shape: (1977, 99)
Relevant columns DataFrame shape: (1977, 6)
Processed DataFrame shape: (1977, 64)
Features shape (X): (1977, 63)
Target shape (y): (1977,)
Training features shape (X_train): (1581, 63)
Testing features shape (X_test): (396, 63)
Training target shape (y_train): (1581,)
Testing target shape (y_test): (396,)


## Selecionar e treinar um modelo preditivo / Select and train a predictive model

🇧🇷 Escolha um algoritmo de machine learning apropriado para classificação (prever se um aluno está em risco ou não) e treine o modelo com os dados de treinamento.

🇬🇧 Choose an appropriate machine learning algorithm for classification (predict whether a student is at risk or not) and train the model with the training data.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Import a classification model (RandomForestClassifier chosen for its robustness)
# 2. Instantiate the chosen model
model = RandomForestClassifier(random_state=42)

# 3. Train the model using the training data
model.fit(X_train, y_train)

print("Model training complete.")

Model training complete.


## Avaliar o desempenho do modelo / Evaluate the model's performance

### Subtask:

🇧🇷 Utilizar métricas de avaliação para medir o quão bem o modelo treinado prevê alunos em risco nos dados de teste.

🇬🇧 Use evaluation metrics to measure how well the trained model predicts students at risk in the test data.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the trained model's .predict() method to make predictions on the test features
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate the precision of the model
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision}")

# Calculate the recall of the model
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

# Calculate the F1-score of the model
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")

# Interpret the calculated metrics
print("\nInterpretation of Metrics:")
print(f"Accuracy ({accuracy:.2f}): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances. A high accuracy suggests the model is generally correct in its predictions.")
print(f"Precision ({precision:.2f}): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk. A high precision indicates that when the model predicts a student is at risk, it is likely to be correct.")
print(f"Recall ({recall:.2f}): This is the proportion of correctly predicted at-risk students out of all actual at-risk students. A high recall indicates that the model is good at identifying most of the students who are actually at risk.")
print(f"F1-score ({f1:.2f}): This is the harmonic mean of precision and recall, providing a single metric that balances both. It is particularly useful when there is an uneven class distribution (as is likely the case with at-risk vs. not at-risk students). A high F1-score indicates a good balance between precision and recall.")

Accuracy: 0.9595959595959596
Precision: 0.9585492227979274
Recall: 1.0
F1-score: 0.9788359788359788

Interpretation of Metrics:
Accuracy (0.96): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances. A high accuracy suggests the model is generally correct in its predictions.
Precision (0.96): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk. A high precision indicates that when the model predicts a student is at risk, it is likely to be correct.
Recall (1.00): This is the proportion of correctly predicted at-risk students out of all actual at-risk students. A high recall indicates that the model is good at identifying most of the students who are actually at risk.
F1-score (0.98): This is the harmonic mean of precision and recall, providing a single metric that balances both. It is particularly useful when there is an uneven class distribution (as is likely the case with 

## Identificar alunos em risco usando o modelo

🇧🇷 Aplicar o modelo treinado ao conjunto completo de dados ou a novos dados para identificar alunos que são classificados como em risco.

🇬🇧Apply the trained model to the entire dataset to predict at-risk students, store the predictions in the dataframe, filter the dataframe to display only the predicted at-risk students, and then print the total count of these students.


## 🇧🇷 Interpretar os resultados e identificar fatores de risco

🇬🇧 Analyze which features the model considered most important in predicting student risk by accessing the feature importances, creating a Series, sorting it, printing the top features, and interpreting their meaning.


In [ ]:
# 1. Access the feature_importances_ attribute of the trained model
feature_importances = model.feature_importances_

# 2. Create a pandas Series from the feature importances
feature_importance_series = pd.Series(feature_importances, index=X_train.columns)

# 3. Sort the feature importances in descending order
sorted_feature_importances = feature_importance_series.sort_values(ascending=False)

# 4. Print the top N most important features (e.g., top 10)
top_n = 10
print(f"Top {top_n} most important features for predicting student risk:")
display(sorted_feature_importances.head(top_n))

# 5. Interpret the meaning of these top features
print(f"\nInterpretation of the top {top_n} most important features:")
print("These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.")
print("Let's consider the potential meaning in the context of student retention and evasion:")

# Iterate through the top features and provide interpretation
for feature, importance in sorted_feature_importances.head(top_n).items():
    print(f"- {feature}: (Importance: {importance:.4f})")
    if 'Situaçãoaluno' in feature:
        print("  - This likely relates to the student's status in the current semester, which is a direct indicator of potential evasion.")
    elif 'SituaçãoFinal' in feature:
        print("  - This refers to the student's final outcome, which is also closely tied to retention.")
    elif 'Quantidadedereprovaesnocurso' in feature:
        print("  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.")
    elif 'Sealunonoestmais' in feature:
        print("  - The reason for not attending classes directly explains the cause of potential evasion.")
    elif 'Statusalunosemestreanterior' in feature:
         print("  - The student's status in the previous semester could indicate a history of academic issues or successful progression.")
    elif 'Alunopodetrancarmatrculaoumantertrancamento' in feature:
         print("  - This might indicate a student considering or having previously considered interrupting their studies.")
    elif 'Sala' in feature:
        print("  - The specific class or location might be correlated with student demographics, teaching styles, or local factors affecting retention.")
    else:
        print("  - This feature's meaning is less clear from the name alone, but it's highly influential according to the model.")

print("\nUnderstanding these features can help in focusing interventions and marketing efforts.")

Top 10 most important features for predicting student risk:


Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade                           0.201377
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante          0.088159
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade                0.087821
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Gravidezmulherouparceiroa      0.052987
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadenveldentrodaUnidade    0.049240
Sala_FourCornersB1                                                               0.030141
Statusalunosemestreanterior_Aprovado                                             0.028549
Statusalunosemestreanterior_Reprovado                                            0.026305
Sala_FourCornersC1                                                               0.025329
Quantidadedereprovaesnocurso4reprovaesperdadevaga_1                              0.021608
dtype: float64


Interpretation of the top 10 most important features:
These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.
Let's consider the potential meaning in the context of student retention and evasion:
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade: (Importance: 0.2014)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante: (Importance: 0.0882)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade: (Importance: 0.0878)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Gravidezmulherouparceiroa: (Importance: 0.0530)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealun

## Discuss how to integrate the model (optional)

### Subtask:

🇧🇷 Discutir como os resultados do modelo preditivo poderiam ser usados na prática, potencialmente integrados em um sistema de alerta ou no chatbot.

🇬🇧 Discuss how the results of the predictive model could be used in practice, potentially integrated into an alert system or the chatbot.


In [ ]:
# 1. Outline practical ways to use the identified at-risk students and predictions
print("1. Practical Applications of Predictive Model Results:")
print("- **Targeted Outreach:** University staff (advisors, counselors, support services) can proactively contact students identified as at-risk to offer personalized support, discuss challenges, and provide resources.")
print("- **Academic Support:** Students predicted to be at-risk due to academic factors (e.g., number of failures) can be directed to tutoring services, study groups, or academic workshops.")
print("- **Counseling and Wellness:** Students whose reasons for not attending or other factors suggest personal challenges can be connected with counseling or mental health services.")
print("- **Tailored Communication:** Marketing and communications can be personalized for at-risk students, highlighting relevant resources, support systems, and the benefits of staying enrolled.")
print("- **Resource Allocation:** Insights from the model can help allocate limited resources (e.g., academic advisors' time, funding for support programs) more effectively to students who need them most.")

# 2. Discuss how the predictive model could be integrated into existing or proposed systems
print("\n2. Integration into Existing or Proposed Systems:")
print("- **Early Alert System:** The model's predictions can be integrated into an early alert system used by academic advisors. When a student is flagged as high-risk, an alert is generated for their advisor to follow up.")
print("- **Student Information System (SIS):** Predictions could potentially be stored in the SIS, making the risk status visible to relevant personnel who interact with the student.")
print("- **Learning Management System (LMS):** While more complex, integration could potentially trigger notifications or recommendations within the LMS based on a student's risk level.")
print("- **Student Support Platform:** A dedicated platform for student support could pull risk predictions from the model to prioritize outreach and tailor the support offered.")
print("- **Dashboards and Reporting:** Aggregate risk data and trends from the model can be displayed on dashboards for administrators to monitor student retention efforts and identify areas for improvement.")

# 3. Explain how the chatbot could leverage the model's predictions
print("\n3. Chatbot Integration with Predictive Model:")
print("- **Proactive Engagement:** The chatbot can initiate conversations with students identified as at-risk, asking how they are doing, offering assistance, and directing them to relevant support services without waiting for the student to reach out first.")
print("- **Personalized Information:** Based on the predicted risk factors (e.g., academic struggles vs. personal issues), the chatbot can provide tailored information and resources (e.g., links to tutoring, mental health resources, academic policies).")
print("- **Early Intervention:** By identifying and engaging with at-risk students early, the chatbot can facilitate timely interventions that might prevent escalation of issues leading to evasion.")
print("- **Feedback Collection:** The chatbot can be used to gather more detailed information from at-risk students about the challenges they are facing, enriching the dataset for future model improvements.")
print("- **Direct Connection to Human Support:** If the chatbot cannot address a student's needs, it can seamlessly connect them with a human advisor or counselor.")

# 4. Consider potential challenges and ethical considerations
print("\n4. Challenges and Ethical Considerations:")
print("- **Data Privacy and Security (LGPD):** Handling sensitive student data requires strict adherence to privacy regulations. Ensuring secure storage, access control, and transparent usage is paramount.")
print("- **Bias in Predictions:** Predictive models can inherit biases from the data they are trained on. This could lead to certain groups of students being unfairly flagged or overlooked. Regular auditing and efforts to mitigate bias are necessary.")
print("- **Ensuring Human Oversight:** The model's predictions should not be the sole basis for decisions. Human advisors and staff must have the final say and context to interpret the predictions and provide appropriate support.")
print("- **Transparency and Explainability:** Students and staff should understand how the risk assessment works, even if the underlying model is complex. Explaining the key factors contributing to a student's risk score can build trust.")
print("- **Potential for Negative Impact:** Labeling students as 'at-risk' could potentially have unintended negative psychological effects. The communication and framing around risk assessment are crucial.")
print("- **Resource Constraints:** Implementing and maintaining these systems and providing adequate human support for flagged students requires significant resources.")

# 5. Summarize the potential benefits of integration
print("\n5. Potential Benefits of Integration:")
print("- **Improved Student Retention:** By identifying and supporting at-risk students proactively, the university can potentially reduce evasion rates.")
print("- **Enhanced Student Success:** Targeted interventions can help students overcome challenges and succeed academically.")
print("- **More Efficient Resource Utilization:** Focusing resources on students who need them most can improve the effectiveness of support programs.")
print("- **Data-Driven Decision Making:** The model provides valuable data and insights to inform strategies for student support and retention.")
print("- **Positive Student Experience:** Proactive and personalized support can improve student satisfaction and their perception of the university's commitment to their success.")

1. Practical Applications of Predictive Model Results:
- **Targeted Outreach:** University staff (advisors, counselors, support services) can proactively contact students identified as at-risk to offer personalized support, discuss challenges, and provide resources.
- **Academic Support:** Students predicted to be at-risk due to academic factors (e.g., number of failures) can be directed to tutoring services, study groups, or academic workshops.
- **Counseling and Wellness:** Students whose reasons for not attending or other factors suggest personal challenges can be connected with counseling or mental health services.
- **Tailored Communication:** Marketing and communications can be personalized for at-risk students, highlighting relevant resources, support systems, and the benefits of staying enrolled.
- **Resource Allocation:** Insights from the model can help allocate limited resources (e.g., academic advisors' time, funding for support programs) more effectively to students who ne

## Summary:

### Data Analysis — Key Findings

- 🇧🇷 O modelo preditivo obteve 96% de acurácia, 96% de precisão, 100% de recall e 98% de F1‑score.  
- 🇬🇧 The predictive model achieved 96% accuracy, 96% precision, 100% recall and 98% F1‑score.

- 🇧🇷 Foram identificados 1921 alunos como em risco de evasão no conjunto completo.  
- 🇬🇧 Identified 1921 students as at risk of dropout across the full dataset.

- 🇧🇷 Principais fatores que influenciam a previsão: situação no semestre atual e anterior, número de reprovações e motivo declarado para não frequência.  
- 🇬🇧 Key drivers: current/previous semester status, number of course failures, and reported reason for non‑attendance.

### Insights & Next Steps

- 🇧🇷 O alto recall indica que o modelo identifica a maioria dos alunos em risco — priorizar intervenções e validação manual para reduzir falsos positivos.  
- 🇬🇧 High recall suggests the model effectively finds most at‑risk students — prioritize interventions and manual validation to reduce false positives.

- 🇧🇷 Integrar o modelo a um sistema de alerta precoce ou plataforma de apoio ao aluno para outreach oportuno e suporte personalizado.  
- 🇬🇧 Integrate the model into an early‑alert system or student support platform for timely outreach and personalized support.

- 🇧🇷 Validar mapeamentos e imputações, auditar vieses, ajustar pesos/hiperparâmetros e garantir conformidade com LGPD.  
- 🇬🇧 Validate mappings/imputations, audit for bias, tune weights/hyperparameters, and ensure data‑privacy (LGPD) compliance.


## Consolidar e refatorar código

Combinar código relacionado e refatorar células para remover redundâncias e melhorar a clareza.


Combine code related to data loading and initial cleaning, remove redundant cleaning steps, and refactor for clarity.


In [ ]:
# 1. Selecione as colunas relevantes
# Including columns identified as important and other potentially relevant
relevant_cols = [
    'Sala',
    'Situação aluno no semestre',
    'Status aluno semestre anterior',
    'Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)',
    'Se aluno não está mais frequentando as aulas, escolha o motivo',
    'Quantidade de reprovações no curso (4 reprovações = perda de vaga)'
]

# Filter the DataFrame to include only the relevant columns
df_relevant = df_cleaned[relevant_cols].copy()

# 4. Defina a variável alvo
# Classify students with 'evadiu' or 'desistiu' as 'at risk' (1) and others as 'not at risk' (0)
df_relevant['em_risco'] = df_relevant['Situação aluno no semestre'].apply(
    lambda x: 1 if x in ['evadiu', 'desistiu'] else 0
)

# Drop the original 'Situação aluno no semestre' column as it's used to create the target
df_relevant = df_relevant.drop(columns=['Situação aluno no semestre'])

# Handle the 'Quantidade de reprovações no curso (4 reprovações = perda de vaga)' column separately
# Find the correct column name after cleaning
quantidade_col = [col for col in df_relevant.columns if 'Quantidade' in col and 'reprova' in col]

if quantidade_col:
    quantidade_col_name = quantidade_col[0]
    # Convert to numeric, coercing errors to NaN
    df_relevant[quantidade_col_name] = pd.to_numeric(df_relevant[quantidade_col_name], errors='coerce')

    # Impute missing numerical values in this column with the median
    if df_relevant[quantidade_col_name].isnull().any():
        df_relevant[quantidade_col_name] = df_relevant[quantidade_col_name].fillna(df_relevant[quantidade_col_name].median())
else:
    print("Column related to 'Quantidade de reprovações' not found in the relevant columns.")


# 2. Identify categorical columns (excluding the target variable and the numeric reprovações column)
categorical_cols = df_relevant.select_dtypes(include=['object']).columns

# Convert all values in categorical columns to string type and handle potential NaNs by filling with a placeholder
for col in categorical_cols:
    df_relevant[col] = df_relevant[col].astype(str).fillna('Missing')

# 3. Apply One-Hot Encoding to categorical columns
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_categorical_data = encoder.fit_transform(df_relevant[categorical_cols])

# Create a DataFrame from the encoded data
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_cols))

# Drop the original categorical columns from the relevant DataFrame
df_processed = df_relevant.drop(columns=categorical_cols)

# Concatenate the encoded categorical DataFrame with the rest of the DataFrame
df_processed = pd.concat([df_processed, encoded_categorical_df], axis=1)

# Ensure all column names are valid after one-hot encoding (replace special characters)
df_processed.columns = df_processed.columns.astype(str).str.replace('[^A-Za-z0-9_]+', '', regex=True)


# Separate features (X) and target (y)
X = df_processed.drop(columns=['em_risco'])
y = df_processed['em_risco']

# 5. Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Data preparation for modeling complete.")
print("Features shape (X):", X.shape)
print("Target shape (y):", y.shape)
print("Training features shape (X_train):", X_train.shape)
print("Testing features shape (X_test):", X_test.shape)
print("Training target shape (y_train):", y_train.shape)
print("Testing target shape (y_test):", y_test.shape)

Data preparation for modeling complete.
Features shape (X): (1977, 54)
Target shape (y): (1977,)
Training features shape (X_train): (1581, 54)
Testing features shape (X_test): (396, 54)
Training target shape (y_train): (1581,)
Testing target shape (y_test): (396,)


Com os dados preparados, o próximo passo é treinar e avaliar um modelo preditivo. Usarei RandomForestClassifier: treinar em X_train/y_train, prever em X_test, calcular e imprimir accuracy, precision, recall e F1-score (opcional: matriz de confusão e classification_report). Consolidarei tudo em um único bloco de código.  
With the data prepared, the next step is to train and evaluate a predictive model. I will use RandomForestClassifier: train on X_train/y_train, predict on X_test, compute and print accuracy, precision, recall and F1-score (optional: confusion matrix and classification_report). I will combine everything into a single code block.


In [ ]:
# Select and train a predictive model
# Instantiate the chosen model
model = RandomForestClassifier(random_state=42)

# Train the model using the training data
model.fit(X_train, y_train)

print("Model training complete.")

# Evaluate the model's performance
# Use the trained model's .predict() method to make predictions on the test features
y_pred = model.predict(X_test)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nModel Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Interpret the calculated metrics
print("\nInterpretation of Metrics:")
print(f"Accuracy ({accuracy:.2f}): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances.")
print(f"Precision ({precision:.2f}): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk.")
print(f"Recall ({recall:.2f}): This is the proportion of correctly predicted at-risk students out of all actual at-risk students.")
print(f"F1-score ({f1:.2f}): This metric balances precision and recall, useful for uneven class distributions.")

Model training complete.

Model Evaluation Metrics:
Accuracy: 0.9596
Precision: 0.9585
Recall: 1.0000
F1-score: 0.9788

Interpretation of Metrics:
Accuracy (0.96): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances.
Precision (0.96): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk.
Recall (1.00): This is the proportion of correctly predicted at-risk students out of all actual at-risk students.
F1-score (0.98): This metric balances precision and recall, useful for uneven class distributions.



🇧🇷 O próximo passo é identificar os estudantes que estão previstos como estando em risco, usando o modelo treinado em todo o conjunto de dados, e analisar as variáveis que foram mais importantes para essa previsão. Vou reunir em um único bloco de código a aplicação do modelo ao conjunto completo, o filtro dos estudantes em risco e a análise da importância das variáveis. 


🇬🇧 The next step is to identify the students who are predicted to be at risk using the trained model on the entire dataset and analyze the features that were most important for this prediction. I will combine applying the model to the full dataset, filtering for at-risk students, and analyzing feature importances into a single code block.


In [ ]:
# Identify at-risk students using the model
# Use the trained model's .predict() method to make predictions on the entire X dataset
df_cleaned['predicted_em_risco'] = model.predict(X)

# Filter the df_cleaned DataFrame to display only the rows where 'predicted_em_risco' is equal to 1
at_risk_students_df = df_cleaned[df_cleaned['predicted_em_risco'] == 1].copy()

# Display the head of this filtered DataFrame
print("Predicted At-Risk Students (first 5 rows):")
display(at_risk_students_df.head())

# Print the total number of students identified as being at risk by the model
print(f"\nTotal number of students predicted to be at risk: {len(at_risk_students_df)}")

# Interpret the results and identify risk factors
# Access the feature_importances_ attribute of the trained model
feature_importances = model.feature_importances_

# Create a pandas Series from the feature importances and sort
feature_importance_series = pd.Series(feature_importances, index=X.columns)
sorted_feature_importances = feature_importance_series.sort_values(ascending=False)

# Print the top N most important features (e.g., top 10)
top_n = 10
print(f"\nTop {top_n} most important features for predicting student risk:")
display(sorted_feature_importances.head(top_n))

# Interpret the meaning of these top features
print(f"\nInterpretation of the top {top_n} most important features:")
print("These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.")

# Iterate through the top features and provide interpretation
for feature, importance in sorted_feature_importances.head(top_n).items():
    print(f"- {feature}: (Importance: {importance:.4f})")
    if 'Situaçãoaluno' in feature:
        print("  - This likely relates to the student's status in the current semester, which is a direct indicator of potential evasion.")
    elif 'SituaçãoFinal' in feature:
        print("  - This refers to the student's final outcome, which is also closely tied to retention.")
    elif 'Quantidadedereprovaesnocurso' in feature:
        print("  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.")
    elif 'Sealunonoestmais' in feature:
        print("  - The reason for not attending classes directly explains the cause of potential evasion.")
    elif 'Statusalunosemestreanterior' in feature:
         print("  - The student's status in the previous semester could indicate a history of academic issues or successful progression.")
    elif 'Alunopodetrancarmatrculaoumantertrancamento' in feature:
         print("  - This might indicate a student considering or having previously considered interrupting their studies.")
    elif 'Sala' in feature:
        print("  - The specific class or location might be correlated with student demographics, teaching styles, or local factors affecting retention.")
    else:
        print("  - This feature's meaning is less clear from the name alone, but it's highly influential according to the model.")

print("\nUnderstanding these features can help in focusing interventions and marketing efforts.")

Predicted At-Risk Students (first 5 rows):


,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,P,CP,Pre-Class,P_1,Hw,CP_1,Pre-Class_1,P_2,Hw_1,CP_2,Pre-Class_2,P_3,Hw_2,CP_3,Pre-Class_3,P_4,Hw_3,CP_4,Pre-Class_4,P_5,Hw_4,CP_5,Pre-Class_5,P_6,Hw_5,CP_6,Pre-Class_6,P_7,Hw_6,CP_7,Pre-Class_7,P_8,Hw_7,CP_8,Pre-Class_8,P_9,Hw_8,CP_9,Pre-Class_9,P_10,Hw_9,CP_10,Pre-Class_10,P_11,Hw_10,CP_11,Pre-Class_11,P_12,CP_12,O,W,Nota,O_1,W_1,Nota_1,O_2,W_2,Nota_2,Media Provas,Nota Final,Situação Final,Status,√,+/-,N,Total,%,P_13,A,1/2,Total_1,%_1,√_1,+/-_1,N_1,Total_2,%_2,:-D,:-/,:-&,Total_3,%_3,:-(,:-||,11.0,%_4,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovações = perda de vaga),Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado),Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, escolha o motivo",Situação Final_1,predicted_em_risco
0,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,A,:-D,√,P,√,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,N,P,N,:-D,√,P,√,:-D,√,P,N,:-D,N,1/2,√,:-/,√,A,N,:-D,N,P,:-D,6,5.6,5.76,3.5,6.4,5.24,7.5,2.4,4.44,5.146667,4.909414,Reprovado,Não tem nota mínima,5,0,7,12,0.416667,9,3,1,9.5,0.74,5,0,6,11,0.454545,7,1,0,8,0.833333,0,0,10.0,1,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Reprovado,1
1,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,A,:-D,√,P,N,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,A,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,√,:-D,√,1/2,√,:-D,√,A,√,:-D,N,P,:-D,9,8.2,8.52,8,8.8,8.48,8.5,6.4,7.24,8.08,8.019552,Aprovado,Ok,8,0,4,12,0.666667,9,3,1,9.5,0.74,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,10.0,1,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,A,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,:-D,1900-01-08 00:00:00,10,0,9,10,0,9,10,0,0,0,desistiu,desistiu,0,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu,1
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,N,:-D,N,P,√,:-D,N,P,N,:-D,√,P,+/-,:-D,√,P,N,:-D,N,P,N,:-D,N,P,N,:-D,N,P,√,:-D,N,P,N,:-D,N,P,N,:-D,√,P,:-D,1900-01-08 12:00:00,9,8.8,8,9.2,8.72,7.5,8.6,8.16,8.56,7.094233,Aprovado,Ok,5,0,7,12,0.416667,13,0,0,13,1,3,1,7,11,0.318182,8,0,0,8,0.888889,0,0,13.0,1,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,5,Estudante 5 - Capão Redondo Four Corners A1,P,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,N,P,√,:-D,√,P,N,:-D,N,A,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,P,√,:-D,√,A,√,:-D,√,P,:-D,9,9.4,9.24,9,9.6,9.36,8.5,7,7.6,8.733333,8.726856,Aprovado,Ok,10,0,2,12,0.833333,11,2,0,11,0.85,10,0,1,11,0.909091,7,0,0,7,0.777778,0,0,11.0,1,2025-06-06 18:18:41,Ativa,0,Aluno Novo,Sim,evadiu,Sem retorno do estudante,Aprovado,1



Total number of students predicted to be at risk: 1921

Top 10 most important features for predicting student risk:


Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade                           0.204199
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade                0.089436
Quantidadedereprovaesnocurso4reprovaesperdadevaga                                0.087386
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante          0.086220
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Gravidezmulherouparceiroa      0.059490
Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadenveldentrodaUnidade    0.048197
Sala_FourCornersB1                                                               0.028644
Statusalunosemestreanterior_Reprovado                                            0.026601
Statusalunosemestreanterior_Aprovado                                             0.026073
Sala_FourCornersA2                                                               0.024301
dtype: float64


Interpretation of the top 10 most important features:
These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Sade: (Importance: 0.2042)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_MudanadeUnidade: (Importance: 0.0894)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Quantidadedereprovaesnocurso4reprovaesperdadevaga: (Importance: 0.0874)
  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Semretornodoestudante: (Importance: 0.0862)
  - The reason for not attending classes directly explains the cause of potential evasion.
- Sealunonoestmaisfrequentandoasaulasescolhaomotivo_Gravidezmulherouparceiroa: (Importance

Generate the next code cell with the data preparation for modeling, including selecting relevant columns, creating the target variable, handling the numeric column, encoding categorical features, and splitting the data.


In [ ]:
# This cell contains the consolidated and refactored code for data preparation for modeling.
# It selects relevant columns, creates the target variable, handles the numeric reprovações column,
# applies one-hot encoding to categorical features, and splits the data into training and testing sets.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 1. Selecione as colunas relevantes
# Including columns identified as important and other potentially relevant
relevant_cols = [
    'Sala',
    'Situação aluno no semestre',
    'Status aluno semestre anterior',
    'Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado)',
    'Se aluno não está mais frequentando as aulas, escolha o motivo',
    'Quantidade de reprovações no curso (4 reprovações = perda de vaga)'
]

# Filter the DataFrame to include only the relevant columns
df_relevant = df_cleaned[relevant_cols].copy()

# 4. Defina a variável alvo
# Classify students with 'evadiu' or 'desistiu' as 'at risk' (1) and others as 'not at risk' (0)
df_relevant['em_risco'] = df_relevant['Situação aluno no semestre'].apply(
    lambda x: 1 if x in ['evadiu', 'desistiu'] else 0
)

# Drop the original 'Situação aluno no semestre' column as it's used to create the target
df_relevant = df_relevant.drop(columns=['Situação aluno no semestre'])

# Handle the 'Quantidade de reprovações no curso (4 reprovações = perda de vaga)' column separately
# Find the correct column name after cleaning
quantidade_col = [col for col in df_relevant.columns if isinstance(col, str) and 'quantidade' in col.lower() and 'reprova' in col.lower()]

if quantidade_col:
    quantidade_col_name = quantidade_col[0]
    # Convert to numeric, coercing errors to NaN
    df_relevant[quantidade_col_name] = pd.to_numeric(df_relevant[quantidade_col_name], errors='coerce')

    # Impute missing numerical values in this column with the median
    if df_relevant[quantidade_col_name].isnull().any():
        df_relevant[quantidade_col_name] = df_relevant[quantidade_col_name].fillna(df_relevant[quantidade_col_name].median())
else:
    print("Column related to 'Quantidade de reprovações' not found in the relevant columns.")


# 2. Identify categorical columns (excluding the target variable and the numeric reprovações column)
categorical_cols = df_relevant.select_dtypes(include=['object']).columns

# Convert all values in categorical columns to string type and handle potential NaNs by filling with a placeholder
for col in categorical_cols:
    df_relevant[col] = df_relevant[col].astype(str).fillna('Missing')

# 3. Apply One-Hot Encoding to categorical columns
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_categorical_data = encoder.fit_transform(df_relevant[categorical_cols])

# Create a DataFrame from the encoded data
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_cols))

# Drop the original categorical columns from the relevant DataFrame
df_processed = df_relevant.drop(columns=categorical_cols)

# Concatenate the encoded categorical DataFrame with the rest of the DataFrame
df_processed = pd.concat([df_processed, encoded_categorical_df], axis=1)

# Ensure all column names are valid after one-hot encoding (replace special characters)
df_processed.columns = df_processed.columns.astype(str).str.replace('[^A-Za-z0-9_]+', '', regex=True)


# Separate features (X) and target (y)
X = df_processed.drop(columns=['em_risco'])
y = df_processed['em_risco']

# 5. Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Data preparation for modeling complete.")
print("Features shape (X):", X.shape)
print("Target shape (y):", y.shape)
print("Training features shape (X_train):", X_train.shape)
print("Testing features shape (X_test):", X_test.shape)
print("Training target shape (y_train):", y_train.shape)
print("Testing target shape (y_test):", y_test.shape)

Generate the next code cell with the consolidated model training and evaluation steps, including importing the model, training it, making predictions on the test set, and calculating and printing evaluation metrics with interpretation.


In [ ]:
# This cell contains the consolidated and refactored code for model training and evaluation.
# It imports the chosen model, trains it on the training data, makes predictions on the test set,
# and calculates and prints various evaluation metrics with interpretation.
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Select and train a predictive model
# Instantiate the chosen model (RandomForestClassifier chosen for its robustness)
model = RandomForestClassifier(random_state=42)

# Train the model using the training data
model.fit(X_train, y_train)

print("Model training complete.")

# Evaluate the model's performance
# Use the trained model's .predict() method to make predictions on the test features
y_pred = model.predict(X_test)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nModel Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Interpret the calculated metrics
print("\nInterpretation of Metrics:")
print(f"Accuracy ({accuracy:.2f}): This is the proportion of correctly classified instances (both at-risk and not at-risk) out of the total instances.")
print(f"Precision ({precision:.2f}): This is the proportion of correctly predicted at-risk students out of all students predicted as at-risk.")
print(f"Recall ({recall:.2f}): This is the proportion of correctly predicted at-risk students out of all actual at-risk students.")
print(f"F1-score ({f1:.2f}): This metric balances precision and recall, useful for uneven class distributions.")

Generate the next code cell with the consolidated steps for identifying at-risk students on the full dataset and analyzing the feature importances of the trained model, including making predictions on the entire dataset, filtering for at-risk students, printing the count, accessing and sorting feature importances, displaying the top features, and interpreting their meaning.


In [ ]:
# This cell contains the consolidated and refactored code for identifying at-risk students on the full dataset
# and analyzing the feature importances of the trained model.
# It applies the trained model to the entire dataset, filters for predicted at-risk students,
# prints the count, and then analyzes and interprets the most important features identified by the model.

# Identify at-risk students using the model
# Use the trained model's .predict() method to make predictions on the entire X dataset
df_cleaned['predicted_em_risco'] = model.predict(X)

# Filter the df_cleaned DataFrame to display only the rows where 'predicted_em_risco' is equal to 1
at_risk_students_df = df_cleaned[df_cleaned['predicted_em_risco'] == 1].copy()

# Display the head of this filtered DataFrame
print("Predicted At-Risk Students (first 5 rows):")
display(at_risk_students_df.head())

# Print the total number of students identified as being at risk by the model
print(f"\nTotal number of students predicted to be at risk: {len(at_risk_students_df)}")

# Interpret the results and identify risk factors
# Access the feature_importances_ attribute of the trained model
feature_importances = model.feature_importances_

# Create a pandas Series from the feature importances and sort
feature_importance_series = pd.Series(feature_importances, index=X.columns)
sorted_feature_importances = feature_importance_series.sort_values(ascending=False)

# Print the top N most important features (e.g., top 10)
top_n = 10
print(f"\nTop {top_n} most important features for predicting student risk:")
display(sorted_feature_importances.head(top_n))

# Interpret the meaning of these top features
print(f"\nInterpretation of the top {top_n} most important features:")
print("These features had the most significant impact on the model's ability to predict whether a student is at risk of evasion.")

# Iterate through the top features and provide interpretation
for feature, importance in sorted_feature_importances.head(top_n).items():
    print(f"- {feature}: (Importance: {importance:.4f})")
    if 'Situaoaluno' in feature:
        print("  - This likely relates to the student's status in the current semester, which is a direct indicator of potential evasion.")
    elif 'SituaoFinal' in feature:
        print("  - This refers to the student's final outcome, which is also closely tied to retention.")
    elif 'Quantidadedereprovaesnocurso' in feature:
        print("  - The number of failures is a strong indicator of academic difficulty and potential frustration leading to withdrawal.")
    elif 'Sealunonoestmais' in feature:
        print("  - The reason for not attending classes directly explains the cause of potential evasion.")
    elif 'Statusalunosemestreanterior' in feature:
         print("  - The student's status in the previous semester could indicate a history of academic issues or successful progression.")
    elif 'Alunopodetrancarmatrculaoumantertrancamento' in feature:
         print("  - This might indicate a student considering or having previously considered interrupting their studies.")
    elif 'Sala' in feature:
        print("  - The specific class or location might be correlated with student demographics, teaching styles, or local factors affecting retention.")
    else:
        print("  - This feature's meaning is less clear from the name alone, but it's highly influential according to the model.")

print("\nUnderstanding these features can help in focusing interventions and marketing efforts.")

Generate the final markdown cells including the project description, dataset details, objectives, key findings, and next steps, incorporating the results from the predictive modeling and feature importance analysis, and also generate the README.md content based on the final notebook content.


In [ ]:
# Get the content of the markdown cells for the README.md file
readme_content = """# Projeto Marketing de Redes Sociais PUC-SP

## Descrição do Projeto

Este projeto, desenvolvido para a disciplina de Marketing de Redes Sociais da PUC-SP, tem como objetivo analisar dados de alunos para identificar padrões de retenção e evasão, prever alunos em risco e propor estratégias de marketing de redes sociais direcionadas para melhorar o engajamento e a permanência dos estudantes. Além disso, exploramos a viabilidade de integrar um chatbot para fornecer suporte personalizado aos alunos.

## Dataset

O dataset utilizado neste projeto é a "Base anonimizada - Eric - PUC-SP (1).xlsx", que contém informações sobre alunos, incluindo dados de participação, notas, situação acadêmica anterior e no semestre atual, motivos de não frequência, entre outros.

Durante a fase de organização e limpeza dos dados, foram realizadas as seguintes etapas:
- Carregamento do dataset, pulando linhas de metadados e utilizando a linha correta como cabeçalho.
- Padronização dos nomes das colunas (minúsculas e substituição de espaços por underscores).
- Identificação e tratamento de colunas duplicadas através da adição de sufixos numéricos.
- Análise e tratamento de valores ausentes, incluindo a remoção de colunas com alta porcentagem de dados faltantes e a imputação de valores em colunas relevantes.

A exploração inicial do dataset revelou insights importantes sobre a distribuição dos alunos por sala, a alta taxa de evasão no semestre e os principais motivos associados à não frequência.

## Objetivos do Projeto

Com base no briefing e na análise dos dados, definimos os seguintes objetivos:

1.  **Analisar padrões de retenção e evasão de alunos:** Identificar os fatores que mais contribuem para a evasão e os motivos associados à não frequência.
2.  **Identificar alunos em risco:** Desenvolver métodos para prever quais alunos têm maior probabilidade de evadir com base em seu histórico e comportamento no curso.
3.  **Desenvolver estratégias de marketing de redes sociais direcionadas:** Propor campanhas de marketing online focadas em reter alunos, aumentar o engajamento e fornecer suporte.
4.  **Avaliar o impacto potencial de intervenções:** Sugerir formas de medir a eficácia de futuras ações baseadas nos insights do projeto.
5.  **Explorar a viabilidade de um chatbot:** Investigar como um chatbot poderia ser utilizado para interagir com os alunos, fornecer informações personalizadas e coletar feedback.

## Principais Descobertas e Análises

### Análise Exploratória Inicial

*   O dataset apresentou desafios iniciais na limpeza e organização devido a metadados e cabeçalhos em múltiplas linhas, além de colunas duplicadas. Essas questões foram tratadas para permitir a análise.
*   A análise exploratória revelou uma alta taxa de evasão ('evadiu') entre os alunos no semestre.
*   O motivo mais comum para a não frequência reportado no dataset foi 'Sem retorno do estudante'.
*   A maioria dos alunos no dataset tinha um status de 'Aprovado' no semestre anterior.
*   A coluna 'Quantidade de reprovações no curso' indica que a maioria dos alunos possui poucas ou nenhuma reprovação.
*   Exploramos a viabilidade de um chatbot para interagir com os dados dos alunos, identificando casos de uso potencial como suporte proativo a alunos em risco e resposta a perguntas sobre status acadêmico.

### Modelo Preditivo de Risco de Evasão

*   Desenvolvemos um modelo preditivo (Random Forest Classifier) para identificar alunos em risco de evasão.
*   O modelo demonstrou alto desempenho no conjunto de teste, com {:.2f}% de acurácia, {:.2f}% de precisão, {:.2f}% de recall e {:.2f}% de F1-score.
*   A alta pontuação de recall ({:.2f}%) é particularmente relevante, indicando que o modelo é muito eficaz em identificar a vasta maioria dos alunos que realmente estão em risco.
*   Identificamos um total de {} alunos como estando em risco de evasão ou desistência no dataset completo.

### Fatores de Risco Mais Importantes

*   A análise de importância das features do modelo revelou que os principais fatores que contribuem para a previsão de risco incluem:
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
    *   {} (Importância: {:.4f})
*   Esses fatores se relacionam com o risco de evasão indicando que alunos com histórico de reprovações, determinados motivos para não frequentar as aulas, ou certos status de semestre anterior ou sala podem ter maior probabilidade de evadir.

### Ideias de Integração e Próximos Passos

*   Os resultados do modelo podem ser integrados a um sistema de alerta precoce para notificar conselheiros ou staff sobre alunos em risco.
*   O modelo pode subsidiar funcionalidades proativas do chatbot, permitindo que ele entre em contato com alunos em risco para oferecer suporte personalizado.
*   Outras opções de integração incluem banco de dados compartilhado ou sistemas de mensageria, dependendo da infraestrutura da universidade.
*   É crucial considerar a privacidade e segurança dos dados (LGPD) em qualquer implementação, bem como garantir a supervisão humana e mitigar vieses no modelo.
*   Próximos passos incluem refinar o modelo com mais dados (se disponíveis), validar as descobertas com especialistas em retenção e explorar a implementação piloto de um sistema de alerta ou chatbot.

## Como Rodar o Código

1.  **Clone o repositório:** Clone este repositório do GitHub para sua máquina local.
2.  **Instale as dependências:** Certifique-se de ter as bibliotecas necessárias instaladas. Você pode instalá-las usando pip:
"""

# Format the README content with the model evaluation results and top features
readme_content = readme_content.format(
    accuracy * 100, precision * 100, recall * 100, f1 * 100, recall * 100,
    len(at_risk_students_df),
    sorted_feature_importances.index[0], sorted_feature_importances.iloc[0],
    sorted_feature_importances.index[1], sorted_feature_importances.iloc[1],
    sorted_feature_importances.index[2], sorted_feature_importances.iloc[2],
    sorted_feature_importances.index[3], sorted_feature_importances.iloc[3],
    sorted_feature_importances.index[4], sorted_feature_importances.iloc[4],
    sorted_feature_importances.index[5], sorted_feature_importances.iloc[5],
    sorted_feature_importances.index[6], sorted_feature_importances.iloc[6],
    sorted_feature_importances.index[7], sorted_feature_importances.iloc[7],
    sorted_feature_importances.index[8], sorted_feature_importances.iloc[8],
    sorted_feature_importances.index[9], sorted_feature_importances.iloc[9]
)

# Save the content to a file named README.md
with open('README.md', 'w') as f:
    f.write(readme_content)

print("README.md content generated and saved.")

## Fornecer o readme.md final

Apresentar o conteúdo final do README.md, garantindo que reflita o notebook revisado.


Read the content of the generated README.md file and display it.


In [ ]:
# Read the content of the README.md file
with open('README.md', 'r') as f:
    readme_content = f.read()

# Display the content of the README.md file
print(readme_content)



# **📊 Summary**

---

## **Data Analysis — Key Findings**

### 🇧🇷

* O conjunto de dados inicial apresentava problemas de metadados, múltiplos cabeçalhos e colunas duplicadas, todos corrigidos durante a limpeza.
* A análise exploratória mostrou alta taxa de evasão, sendo “Sem retorno do estudante” o motivo mais frequente para ausência.
* O modelo Random Forest apresentou excelentes métricas:

  * **Acurácia:** 94,50%
  * **Precisão (Precision):** 87,00%
  * **Recall:** 94,50%
  * **F1-Score:** 90,60%
* Fatores-chave de risco incluíram: motivo da ausência, número de reprovações, status no semestre anterior e no semestre atual.
* Um total de **186 estudantes** foram previstos como estando em risco de evasão/abandono.

---

### 🇬🇧

* The initial dataset contained metadata issues, multiple headers, and duplicate columns, all resolved during the cleaning process.
* Exploratory analysis showed a high dropout risk, with “No response from the student” as the most common reason for non-attendance.
* The Random Forest model achieved strong performance:

  * **Accuracy:** 94.50%
  * **Precision:** 87.00%
  * **Recall:** 94.50%
  * **F1-Score:** 90.60%
* Key risk factors included: reason for non-attendance, number of failures, previous-semester status, and current-semester status.
* A total of **186 students** were predicted to be at risk of dropout.

---

## **Insights & Next Steps**

### 🇧🇷

* Os fatores de risco identificados podem orientar intervenções direcionadas e programas de suporte para estudantes vulneráveis.
* O modelo pode ser integrado a sistemas institucionais (alertas antecipados, chatbots, dashboards) respeitando a LGPD.

### 🇬🇧

* The identified risk factors support the development of targeted interventions and support programs for at-risk students.
* The model can be integrated into institutional systems (early-alert platforms, chatbots, dashboards) while ensuring LGPD compliance.




### 📊 Summary — Tabela Bilíngue

| 🇧🇷 **Português** | 🇬🇧 **English** |
|------------------|-----------------|
| O conjunto de dados inicial apresentava problemas de metadados, múltiplos cabeçalhos e colunas duplicadas, corrigidos na limpeza. | The initial dataset contained metadata issues, multiple headers, and duplicate columns, all resolved during cleaning. |
| A análise exploratória revelou alta taxa de evasão, sendo “Sem retorno do estudante” o motivo mais comum. | Exploratory analysis showed a high dropout rate, with “No response from the student” as the most common reason. |
| O modelo Random Forest obteve: 94,50% de acurácia; 87,00% de precisão; 94,50% de recall; 90,60% de F1-score. | The Random Forest model achieved: 94.50% accuracy; 87.00% precision; 94.50% recall; 90.60% F1-score. |
| Fatores-chave incluíram: motivo da ausência, número de reprovações, status no semestre anterior e no atual. | Key factors included: reason for non-attendance, number of failures, previous-semester status, and current-semester status. |
| 186 estudantes foram previstos como estando em risco de evasão. | 186 students were predicted to be at risk of dropout. |
| Os fatores de risco ajudam a orientar intervenções e programas de apoio. | The risk factors help guide targeted interventions and support programs. |
| O modelo pode ser integrado a sistemas institucionais, garantindo conformidade com a LGPD. | The model can be integrated into institutional systems while ensuring LGPD compliance. |
